In [ ]:
import os
import os.path
import tornado
import scipy
import h5py
import sys
import numpy as np
from numpy.matlib import repmat
import pandas as pd
import pylab
import matplotlib.path as mpath
import random
import chaosmagpy as cp
import statsmodels.api as sm
from tqdm import tqdm

import matplotlib.pyplot as plt
from matplotlib.ticker import AutoMinorLocator, MultipleLocator
from statistics import variance
from matplotlib.lines import Line2D
import webgeodyn
import webgeodyn.server
from webgeodyn.models import Models, Model
from webgeodyn.filters import time
from webgeodyn.filters.spectral import keep_m
from webgeodyn.processing import spectra
from webgeodyn.inout import s1fs2, midpath, ced, covobs_x2
from webgeodyn.constants import rE, rC
from webgeodyn.processing import psd #, psd_dev
from cartopy import crs as ccrs

First we need to prepare the coefficients from the IGRF github to be entered into pygeodyn

Function to take the coefficient folders from github.com/IAGA-VMOD/IGRF14eval/tree/main/data/coefficients to put into the 1-year format needed for pygeodyn. Note you will have to change the 'pygeodyn_folder' to be the file of your choice

In [ ]:
pygeodyn_folder='PATH'

def IGRF_hdf5(group):
        
    file = str(group)+'.hdf5'
    path = str(pygeodyn_folder)+'data/observations/IGRF_14_eval/'+str(group)

    try:
        os.mkdir(path)
    except OSError as error:
        print(error)
        #os.remove(path+'/'+file)
    
    if group=='Huber':
        DGRF_group = 'Median'
    else:
        DGRF_group = group
    OTHER_group = group
    
    start_yr=2020
    end_yr=2030
    step=1
    
    l_b=13
    g=np.array(np.loadtxt('Original/DGRF/DGRF_'+str(DGRF_group)+'.cof')[:,2])
    h=np.array(np.loadtxt('Original/DGRF/DGRF_'+str(DGRF_group)+'.cof')[:,3])
    gnm_step=[]
    var_gnm=[]
    k=0
    for l in range(1,l_b+1,1):
        for m in range(0,l+1,1):
            if m==0:
                gnm_step.append(g[k])
                var_gnm.append(0.001)
            else:
                gnm_step.append(g[k])
                var_gnm.append(0.001)
                gnm_step.append(h[k])
                var_gnm.append(0.001)
            k=k+1
    
    g_dot=(np.array(np.loadtxt('Original/IGRF/IGRF_'+str(OTHER_group)+'.cof')[:,2])-np.array(np.loadtxt('Original/DGRF/DGRF_'+str(DGRF_group)+'.cof')[:,2]))/(2025.0-2020.0)
    h_dot=(np.array(np.loadtxt('Original/IGRF/IGRF_'+str(OTHER_group)+'.cof')[:,3])-np.array(np.loadtxt('Original/DGRF/DGRF_'+str(DGRF_group)+'.cof')[:,3]))/(2025.0-2020.0)
    
    g=np.array(np.loadtxt('Original/IGRF/IGRF_'+str(OTHER_group)+'.cof')[:,2])
    h=np.array(np.loadtxt('Original/IGRF/IGRF_'+str(OTHER_group)+'.cof')[:,3])
    
    original_IGRF = []
    dgnm_step=[]
    var_dgnm=[]
    k=0
    for l in range(1,l_b+1,1):
        for m in range(0,l+1,1):
            if m==0:
                dgnm_step.append(g_dot[k])
                var_dgnm.append(0.001)
                original_IGRF.append(g[k])
            else:
                dgnm_step.append(g_dot[k])
                var_dgnm.append(0.001)
                original_IGRF.append(g[k])
                dgnm_step.append(h_dot[k])
                var_dgnm.append(0.001)
                original_IGRF.append(h[k])
            k=k+1
                
    l_sv=8
    g_dot=np.array(np.loadtxt('Original/SV/SV_'+str(OTHER_group)+'.cof')[:,2])
    h_dot=np.array(np.loadtxt('Original/SV/SV_'+str(OTHER_group)+'.cof')[:,3])
    dgnm_step_25=[]
    var_dgnm_step_25=[]
    k=0
    for l in range(1,l_sv+1,1):
        for m in range(0,l+1,1):
            if m==0:
                dgnm_step_25.append(g_dot[k])
                var_dgnm_step_25.append(0.001)
            else:
                dgnm_step_25.append(g_dot[k])
                var_dgnm_step_25.append(0.001)
                dgnm_step_25.append(h_dot[k])
                var_dgnm_step_25.append(0.001)
            k=k+1
    for l in range(l_sv+1, l_b+1,1):
        for m in range(0,l+1,1):
            if m==0:
                dgnm_step_25.append(0.0)
                var_dgnm_step_25.append(100.0)
            else:
                dgnm_step_25.append(0.0)
                dgnm_step_25.append(0.0)
                var_dgnm_step_25.append(100.0)
                var_dgnm_step_25.append(100.0)

    times_array = []
    gnm_array=[]
    dgnm_array = []
    var_gnm_array=[]
    var_dgnm_array=[]
    
    for times in range(start_yr, end_yr+1, step):
        times_array.append(float(times))
        
        gnm_array.append(np.array(gnm_step))
        var_gnm_array.append(np.array(var_gnm))
        
        if times == 2025.0:
            dgnm_step= np.copy(dgnm_step_25)
            var_dgnm_step = np.copy(var_dgnm_step_25)
            # Check IGRF -gnm_step =~ 0
            diff = original_IGRF-gnm_step
            if np.any(diff>0.0000000005):
                print('WARNING DGRF+SV DOES NOT EQUAL IGRF')
            
        dgnm_array.append(np.array(dgnm_step))
        var_dgnm_array.append(np.array(var_dgnm))
        
        gnm_step = np.array(gnm_step)+np.array(dgnm_step)
    
    gnm_realisations = np.zeros((200, 11, 195))
    dgnm_realisations = np.zeros((200, 11, 195))
    gnm_realisations[:] = gnm_array
    dgnm_realisations[:] = dgnm_array
    
    with h5py.File(path+'/'+file, 'w') as f:
        f.create_dataset('dgnm', data=dgnm_realisations)
        f.create_dataset('gnm', data=gnm_realisations)
        f.create_dataset('times', data=times_array)
        f.create_dataset('var_gnm', data=var_gnm_array)
        f.create_dataset('var_dgnm', data=var_dgnm_array)
        
    print('Saved file: ' + str(file))
    

In [ ]:
group_list=['BGS','CSES','GFZ','Huber','IPGP','ISTERRE','MISTA','NOAA','Strasbourg','TU_Berlin','UCM','USTHB','WHU']
for j in range(len(group_list)):
    IGRF_hdf5(group_list[j])    

After downloading the model from https://www.spacecenter.dk/files/magnetic-models/CHAOS-8/ you can then run this to produce the DTU model with a higher SV degree

In [ ]:
# Load CHAOS model at Lsv=13
file = 'DTU_13.hdf5'
start_yr=2020
end_yr=2030
step=1
path = str(pygeodyn_folder)+'data/observations/IGRF_14_eval/DTU13'

l_b=13
g=np.array(np.loadtxt('Original/DGRF/DGRF_'+str('DTU')+'.cof')[:,2])
h=np.array(np.loadtxt('Original/DGRF/DGRF_'+str('DTU')+'.cof')[:,3])
gnm_step=[]
var_gnm=[]
k=0
for l in range(1,l_b+1,1):
    for m in range(0,l+1,1):
        if m==0:
            gnm_step.append(g[k])
            var_gnm.append(0.001)
        else:
            gnm_step.append(g[k])
            var_gnm.append(0.001)
            gnm_step.append(h[k])
            var_gnm.append(0.001)
        k=k+1

g_dot=(np.array(np.loadtxt('Original/IGRF/IGRF_'+str('DTU')+'.cof')[:,2])-np.array(np.loadtxt('Original/DGRF/DGRF_'+str('DTU')+'.cof')[:,2]))/(2025.0-2020.0)
h_dot=(np.array(np.loadtxt('Original/IGRF/IGRF_'+str('DTU')+'.cof')[:,3])-np.array(np.loadtxt('Original/DGRF/DGRF_'+str('DTU')+'.cof')[:,3]))/(2025.0-2020.0)

g=np.array(np.loadtxt('Original/IGRF/IGRF_'+str('DTU')+'.cof')[:,2])
h=np.array(np.loadtxt('Original/IGRF/IGRF_'+str('DTU')+'.cof')[:,3])

original_IGRF = []
dgnm_step=[]
var_dgnm=[]
k=0
for l in range(1,l_b+1,1):
    for m in range(0,l+1,1):
        if m==0:
            dgnm_step.append(g_dot[k])
            var_dgnm.append(0.001)
            original_IGRF.append(g[k])
        else:
            dgnm_step.append(g_dot[k])
            var_dgnm.append(0.001)
            original_IGRF.append(g[k])
            dgnm_step.append(h_dot[k])
            var_dgnm.append(0.001)
            original_IGRF.append(h[k])
        k=k+1

l_sv=13
model = cp.load_CHAOS_matfile(str(pygeodyn_folder)+'data/observations/CHAOS-8/CHAOS-8-1.mat')
time = 2025.0
dgnm_step_25 = model.synth_coeffs_tdep(time, nmax=13, deriv=1)  # shape: (10, 224)
var_dgnm_step_25=[]
k=0
for l in range(1,l_sv+1,1):
    for m in range(0,l+1,1):
        if m==0:
            var_dgnm_step_25.append(0.001)
        else:
            var_dgnm_step_25.append(0.001)
            var_dgnm_step_25.append(0.001)
        k=k+1

times_array = []
gnm_array=[]
dgnm_array = []
var_gnm_array=[]
var_dgnm_array=[]

for times in range(start_yr, end_yr+1, step):
    times_array.append(float(times))

    gnm_array.append(np.array(gnm_step))
    var_gnm_array.append(np.array(var_gnm))

    if times == 2025.0:
        dgnm_step= np.copy(dgnm_step_25)
        var_dgnm_step = np.copy(var_dgnm_step_25)
        # Check IGRF -gnm_step =~ 0
        diff = original_IGRF-gnm_step
        if np.any(diff>0.0000000005):
            print('WARNING DGRF+SV DOES NOT EQUAL IGRF')

    dgnm_array.append(np.array(dgnm_step))
    var_dgnm_array.append(np.array(var_dgnm))

    gnm_step = np.array(gnm_step)+np.array(dgnm_step)
    
gnm_realisations = np.zeros((200, 11, 195))
dgnm_realisations = np.zeros((200, 11, 195))
gnm_realisations[:] = gnm_array
dgnm_realisations[:] = dgnm_array

with h5py.File(path+'/'+file, 'w') as f:
    f.create_dataset('dgnm', data=dgnm_realisations)
    f.create_dataset('gnm', data=gnm_realisations)
    f.create_dataset('times', data=times_array)
    f.create_dataset('var_gnm', data=var_gnm_array)
    f.create_dataset('var_dgnm', data=var_dgnm_array)

print('Saved file: ' + str(file))

After downloading the SV model from https://ionocovar.agnld.uni-potsdam.de/Kalmag/Kalmag/Model/ you can then run this to produce the TU_Berlin model with a higher SV degree

In [ ]:
# Load Kalmag model at Lsv=13
path = str(pygeodyn_folder)+'data/observations/IGRF_14_eval/TU_Berlin_13'

file = 'TU_Berlin_13.hdf5'
start_yr=2020
end_yr=2030
step=1

l_b=13
g=np.array(np.loadtxt('Original/DGRF/DGRF_'+str('TU_Berlin')+'.cof')[:,2])
h=np.array(np.loadtxt('Original/DGRF/DGRF_'+str('TU_Berlin')+'.cof')[:,3])
gnm_step=[]
var_gnm=[]
k=0
for l in range(1,l_b+1,1):
    for m in range(0,l+1,1):
        if m==0:
            gnm_step.append(g[k])
            var_gnm.append(0.001)
        else:
            gnm_step.append(g[k])
            var_gnm.append(0.001)
            gnm_step.append(h[k])
            var_gnm.append(0.001)
        k=k+1

g_dot=(np.array(np.loadtxt('Original/IGRF/IGRF_'+str('TU_Berlin')+'.cof')[:,2])-np.array(np.loadtxt('Original/DGRF/DGRF_'+str('TU_Berlin')+'.cof')[:,2]))/(2025.0-2020.0)
h_dot=(np.array(np.loadtxt('Original/IGRF/IGRF_'+str('TU_Berlin')+'.cof')[:,3])-np.array(np.loadtxt('Original/DGRF/DGRF_'+str('TU_Berlin')+'.cof')[:,3]))/(2025.0-2020.0)

g=np.array(np.loadtxt('Original/IGRF/IGRF_'+str('TU_Berlin')+'.cof')[:,2])
h=np.array(np.loadtxt('Original/IGRF/IGRF_'+str('TU_Berlin')+'.cof')[:,3])

original_IGRF = []
dgnm_step=[]
var_dgnm=[]
k=0
for l in range(1,l_b+1,1):
    for m in range(0,l+1,1):
        if m==0:
            dgnm_step.append(g_dot[k])
            var_dgnm.append(0.001)
            original_IGRF.append(g[k])
        else:
            dgnm_step.append(g_dot[k])
            var_dgnm.append(0.001)
            original_IGRF.append(g[k])
            dgnm_step.append(h_dot[k])
            var_dgnm.append(0.001)
            original_IGRF.append(h[k])
        k=k+1

l_sv=13
dgnm_step_25 = np.loadtxt(str(pygeodyn_folder)+'data/observations/IGRF_14_eval/TU_Berlin_13/SV_original.txt')[1:,1]
var_dgnm_step_25=[]
k=0
for l in range(1,l_sv+1,1):
    for m in range(0,l+1,1):
        if m==0:
            var_dgnm_step_25.append(0.001)
        else:
            var_dgnm_step_25.append(0.001)
            var_dgnm_step_25.append(0.001)
        k=k+1

times_array = []
gnm_array=[]
dgnm_array = []
var_gnm_array=[]
var_dgnm_array=[]

for times in range(start_yr, end_yr+1, step):
    times_array.append(float(times))

    gnm_array.append(np.array(gnm_step))
    var_gnm_array.append(np.array(var_gnm))

    if times == 2025.0:
        dgnm_step= np.copy(dgnm_step_25)
        var_dgnm_step = np.copy(var_dgnm_step_25)
        # Check IGRF -gnm_step =~ 0
        diff = original_IGRF-gnm_step
        if np.any(diff>0.0000000005):
            print('WARNING DGRF+SV DOES NOT EQUAL IGRF')

    dgnm_array.append(np.array(dgnm_step))
    var_dgnm_array.append(np.array(var_dgnm))

    gnm_step = np.array(gnm_step)+np.array(dgnm_step)
    
gnm_realisations = np.zeros((200, 11, 195))
dgnm_realisations = np.zeros((200, 11, 195))
gnm_realisations[:] = gnm_array
dgnm_realisations[:] = dgnm_array

with h5py.File(path+'/'+file, 'w') as f:
    f.create_dataset('dgnm', data=dgnm_realisations)
    f.create_dataset('gnm', data=gnm_realisations)
    f.create_dataset('times', data=times_array)
    f.create_dataset('var_gnm', data=var_gnm_array)
    f.create_dataset('var_dgnm', data=var_dgnm_array)

print('Saved file: ' + str(file))

Optional check for the format for pygeodyn

In [ ]:
with h5py.File(str(pygeodyn_folder)+'data/observations/IGRF_14_eval/BGS/BGS.hdf5', 'r') as f:
    print(list(f))
    print(np.array(f['times'].shape))
    print(np.array(f['dgnm'].shape))
    print(np.array(f['gnm'].shape))
    print(np.array(f['var_gnm'].shape))

Now the coefficients are in the correct format, we can run the models in pygeodyn. Instructions to run the pygeodyn core flow inversion is here: https://geodynamo.gricad-pages.univ-grenoble-alpes.fr/pygeodyn/index.html

An example of a .conf file for the set up given in the paper is here (you will need to add the model name):

You must also add igrfcompare as a new file type. You will need to change the MODEL every new run. See https://geodynamo.gricad-pages.univ-grenoble-alpes.fr/pygeodyn/usage_new_types.html

Repeat the pygeodyn core flow inversion for all candidate models of your choosing. Alternatively you can access the inversion outputs presented in the paper from the zenodo repository where this notebook is found. Below we introduce the plotting and analysis functions used within the paper

Here we define some functions we will use later:

In [ ]:
# Define the SV or ER spectra calculation
def computeSVSpectra(lmax, array2D):
    fac_l = np.zeros((lmax))
    Spec_tmp = np.zeros(lmax)

    k=0
    for j in range(lmax):
        l=j+1
        fac_l[j] = l+1

        tmp=0.
        for m in range (2*l+1):
            tmp = tmp + fac_l[j]*array2D[k]**2
            k=k+1
        Spec_tmp[j] = tmp

    return Spec_tmp

## Load models
PlotModels = {}
LODmodel ={}

def readmodelhdf5(file, model_name, col, mark, linesty):
    with h5py.File(file) as f:
        f_data = f['analysed']
        T = np.array(f_data['times'])
        U =  np.array(f['analysed']['U'])
        SV =  np.array(f['analysed']['SV'])
        MF =  np.array(f['analysed']['MF'])
        ER =  np.array(f['analysed']['ER'])
    Uav = np.mean(U,axis=0)
    SVav = np.mean(SV,axis=0)
    MFav = np.mean(MF,axis=0)
    ERav = np.mean(ER,axis=0)
    
    Ustd = np.std(U, axis=0)
    
    PlotModels[model_name] = dict([('Time', T), ('U', Uav), ('Utot', U), ('Ustd', Ustd), ('SV', SVav), ('MF', MFav), ('ER', ERav), ('Color', col), ('Marker', mark), ('LineStyle', linesty)])
    return PlotModels


# Define the flow and flow diff spectra 
def computeUSpectra(lmax, array2D):
    fac_l = np.zeros((lmax))
    Spec_tmp = np.zeros(lmax)

    k=0
    for j in range(lmax):
        l=j+1
        fac_l[j] = l*(l+1)/(2*l+1)

        tmp=0.
        for m in range (2*l+1):
            tmp = tmp + fac_l[j]*(array2D[k]**2 + array2D[k+(lmax*(lmax+2))]**2)
            k=k+1
        Spec_tmp[j] = tmp

    return Spec_tmp

def correlationcoeff2D(a, b, Nph, Nth):
    '''
    Assume a and b are 2 maps of the same size where we want to calculate the average correlation coefficient
    '''

    corr_map = np.zeros((Nth-1, 1))
    
    for i in range(0,Nth-1):
        tmp = np.corrcoef(a[i], b[i])[0,1]
        corr_map[i] = tmp
    
    corrcoeff = np.mean(corr_map,axis=0)
    return corrcoeff

def readPlotmodelhdf5(file):
    with h5py.File(file) as f:
        f_data = f['analysed']
        T = np.array(f_data['times'])
        U =  np.array(f['analysed']['U'])
        SV =  np.array(f['analysed']['SV'])
        MF =  np.array(f['analysed']['MF'])
        ER =  np.array(f['analysed']['ER'])

    Uav = np.mean(U,axis=0)
    Ustd = np.std(U,axis=0)

    SVav = np.mean(SV,axis=0)
    ERav = np.mean(ER,axis=0)
    MFav = np.mean(MF,axis=0)
    
    f_model = webgeodyn.models.Model()
    f_model.addMeasure(measureName='U',measureType='U',lmax=15,units='km/yr',data=Uav,rmsdata=None,times=T)
    f_model.addMeasure(measureName='Ustd',measureType='U',lmax=15,units='km/yr',data=Ustd,rmsdata=None,times=T)
    f_model.addMeasure(measureName='MF',measureType='MF',lmax=13,units='nT/yr',data=MFav,rmsdata=None,times=T)
    f_model.addMeasure(measureName='SV',measureType='SV',lmax=13,units='nT/yr',data=SVav,rmsdata=None,times=T)
    f_model.addMeasure(measureName='ER',measureType='SV',lmax=13,units='nT/yr',data=ERav,rmsdata=None,times=T)
    return f_model

def compute_huber_weighted_mean(maps, huber_tuning=1.345, verbose=True):
    """
    Computes Huber-weighted mean and weights across models for each pixel.

    Parameters:
    ----------
    maps : np.ndarray
        Input array of shape (num_models, height, width), where each [i, :, :] is a 2D map.
    huber_tuning : float
        Tuning constant for Huber's loss function (default 1.345).
    verbose : bool
        Whether to display a progress bar.

    Returns:
    -------
    huber_mean_map : np.ndarray
        Robust mean per pixel, shape (height, width).
    huber_weights : np.ndarray
        Huber weights per model and pixel, shape (num_models, height, width).
    """
    num_models, H, W = maps.shape
    huber_mean_map = np.zeros((H, W))
    huber_weights = np.zeros((num_models, H, W))

    iterator = tqdm(range(H), desc="Computing Huber stats") if verbose else range(H)

    for i in iterator:
        for j in range(W):
            y = maps[:, i, j]

            # Skip if all values are nan
            if np.all(np.isnan(y)):
                huber_mean_map[i, j] = np.nan
                huber_weights[:, i, j] = np.nan
                continue

            # Remove nan entries
            valid_idx = ~np.isnan(y)
            y_valid = y[valid_idx]
            X = np.ones((len(y_valid), 1))  # intercept-only model

            try:
                rlm_model = sm.RLM(y_valid, X, M=sm.robust.norms.HuberT(t=huber_tuning))
                rlm_results = rlm_model.fit()

                huber_mean_map[i, j] = rlm_results.params[0]

                # Store weights only for valid indices
                weights = rlm_results.weights
                full_weights = np.zeros(num_models)
                full_weights[valid_idx] = weights
                huber_weights[:, i, j] = full_weights

            except Exception as e:
                # Fallback to mean if fitting fails
                huber_mean_map[i, j] = np.nanmean(y)
                huber_weights[:, i, j] = np.where(valid_idx, 1.0, 0.0)

    return huber_mean_map, huber_weights

You can either load models individually like:

In [ ]:
pygeodyn_results = 'PATH'
with h5py.File(str(pygeodyn_results)+'Huber_Current_computation.hdf5') as f:
    f_data = f['analysed']
    T = np.array(f_data['times'])
    U =  np.array(f['analysed']['U'])
    SV =  np.array(f['analysed']['SV'])
    MF =  np.array(f['analysed']['MF'])
    ER =  np.array(f['analysed']['ER'])

Uav = np.mean(U,axis=0)
Ustd = np.std(U,axis=0)

SVav = np.mean(SV,axis=0)
ERav = np.mean(ER,axis=0)
MFav = np.mean(MF,axis=0)

f_model = webgeodyn.models.Model()
f_model.addMeasure(measureName='U',measureType='U',lmax=15,units='km/yr',data=Uav,rmsdata=None,times=T)
f_model.addMeasure(measureName='Ustd',measureType='U',lmax=15,units='km/yr',data=Ustd,rmsdata=None,times=T)
f_model.addMeasure(measureName='MF',measureType='MF',lmax=13,units='nT/yr',data=MFav,rmsdata=None,times=T)
f_model.addMeasure(measureName='SV',measureType='SV',lmax=13,units='nT/yr',data=SVav,rmsdata=None,times=T)
f_model.addMeasure(measureName='ER',measureType='SV',lmax=13,units='nT/yr',data=ERav,rmsdata=None,times=T)

Or load multiple models together like this:

In [ ]:
group_list=['BGS','CSES','DTU','GFZ','Huber','IPGP','ISTERRE','MISTA','NOAA','Strasbourg','TU_Berlin','UCM','USTHB','WHU']

pygeodyn_results = 'PATH'

PlotModels = readmodelhdf5(str(pygeodyn_results)+'BGS_Current_computation.hdf5', 'BGS', 'green', 'o','-')
PlotModels = readmodelhdf5(str(pygeodyn_results)+'CSES_Current_computation.hdf5', 'CSES', 'brown', 'o','-')
PlotModels = readmodelhdf5(str(pygeodyn_results)+'DTU_Current_computation.hdf5', 'DTU', 'red', 'o','-')
PlotModels = readmodelhdf5(str(pygeodyn_results)+'GFZ_Current_computation.hdf5', 'GFZ', 'orange', 'o','-')
PlotModels = readmodelhdf5(str(pygeodyn_results)+'Huber_Current_computation.hdf5', 'HUBER', 'black', 'o','-')
PlotModels = readmodelhdf5(str(pygeodyn_results)+'IPGP_Current_computation.hdf5', 'IPGP', 'yellow', 'o','-')
PlotModels = readmodelhdf5(str(pygeodyn_results)+'ISTERRE_Current_computation.hdf5', 'ISTERRE', 'grey', 'o','-')
PlotModels = readmodelhdf5(str(pygeodyn_results)+'Strasbourg_Current_computation.hdf5', 'ITES', 'cyan', 'o','-')
PlotModels = readmodelhdf5(str(pygeodyn_results)+'MISTA_Current_computation.hdf5', 'MISTA', 'purple', 'o','-')
PlotModels = readmodelhdf5(str(pygeodyn_results)+'NOAA_Current_computation.hdf5', 'NOAA', 'pink', 'o','-')
PlotModels = readmodelhdf5(str(pygeodyn_results)+'TU_Berlin_Current_computation.hdf5', 'TUB', 'blue', 'o','-')
PlotModels = readmodelhdf5(str(pygeodyn_results)+'UCM_Current_computation.hdf5', 'UCM', 'magenta', 'o','-')
PlotModels = readmodelhdf5(str(pygeodyn_results)+'USTHB_Current_computation.hdf5', 'USTHB', 'deeppink', 'o','-')
PlotModels = readmodelhdf5(str(pygeodyn_results)+'WHU_Current_computation.hdf5', 'WHU', 'teal', 'o','-')

Model_list= list(PlotModels)

Model_parameters=list(PlotModels['BGS'])
Model_times=list(PlotModels['BGS']['Time'])

print("List of Models = " + str(Model_list))
print("Parameters of Each Model = " + str(Model_parameters))
print("Model Time = " + str(Model_times))

Figure 2: Spectra of the SV, ER and U for 2020, 2025 and 2030 for all candidates 

In [ ]:
fig_SV, ((ax1, ax2), (ax3, ax4), (ax5, ax6)) = plt.subplots(3, 2, layout='constrained', figsize=(10,12.5))
SV_spec = []
ER_spec = []

for i in range(len(Model_list)): 
    model = Model_list[i]
    lmax = 13
    x_SV = np.array(np.linspace(1,lmax, lmax))
    loc = np.where(PlotModels[model]['Time'] == 2020.0)[0]
    array2D_SV = PlotModels[model]['SV'][loc][0]
    SV_spec = computeSVSpectra(lmax, array2D_SV)
    array2D_ER = PlotModels[model]['ER'][loc][0]
    ER_spec = computeSVSpectra(lmax, array2D_ER)
    
    ax1.plot(x_SV, np.array(SV_spec), label = model, color = PlotModels[model]['Color'], ls = '-', marker = PlotModels[model]['Marker'],  markersize=5, lw=1)
    ax1.plot(x_SV, np.array(ER_spec), label = model, color = PlotModels[model]['Color'], ls = '--', marker = PlotModels[model]['Marker'],  markersize=5, lw=1)
    ax1.set_title('SV and ER in January 2020', fontsize=14)
    #ax1.set_xlabel('Spherical Harmonic Degree', fontsize=12)
    ax1.set_ylabel('Spectral Energy $(nT/yr)^2$', fontsize=12)
    #ax1.set_xticks(np.linspace(2,12, 6))
    ax1.semilogy()
    ax1.axis([0, lmax+1, 0.001, 10000])
    ax1.set_box_aspect(1)
    
    lmax = 8
    x_SV = np.array(np.linspace(1,lmax, lmax))
    loc = np.where(PlotModels[model]['Time'] == 2025.0)[0]
    array2D_SV = PlotModels[model]['SV'][loc][0]
    SV_spec = computeSVSpectra(lmax, array2D_SV)
    array2D_ER = PlotModels[model]['ER'][loc][0]
    ER_spec = computeSVSpectra(lmax, array2D_ER)
    
    ax3.plot(x_SV, np.array(SV_spec), label = model, color = PlotModels[model]['Color'], ls = '-', marker = PlotModels[model]['Marker'],  markersize=5, lw=1)
    ax3.plot(x_SV, np.array(ER_spec), label = model, color = PlotModels[model]['Color'], ls = '--', marker = PlotModels[model]['Marker'],  markersize=5, lw=1)
    ax3.set_title('SV and ER in January 2025', fontsize=14)
    #ax3.set_xlabel('Spherical Harmonic Degree', fontsize=12)
    ax3.set_ylabel('Spectral Energy $(nT/yr)^2$', fontsize=12)
    #ax3.set_xticks(np.linspace(2,12, 6))
    ax3.semilogy()
    ax3.axis([0, 14, 0.001, 10000])
    ax3.set_box_aspect(1)
    
    loc = np.where(PlotModels[model]['Time'] == 2030.0)[0]
    array2D_SV = PlotModels[model]['SV'][loc][0]
    SV_spec = computeSVSpectra(lmax, array2D_SV)
    array2D_ER = PlotModels[model]['ER'][loc][0]
    ER_spec = computeSVSpectra(lmax, array2D_ER)
    
    ax5.plot(x_SV, np.array(SV_spec), label = model, color = PlotModels[model]['Color'], ls = '-', marker = PlotModels[model]['Marker'],  markersize=5, lw=1)
    ax5.plot(x_SV, np.array(ER_spec), label = model, color = PlotModels[model]['Color'], ls = '--', marker = PlotModels[model]['Marker'],  markersize=5, lw=1)
    ax5.set_title('SV and ER in January 2030', fontsize=14)
    ax5.set_xlabel('Spherical Harmonic Degree', fontsize=12)
    ax5.set_ylabel('Spectral Energy $(nT/yr)^2$', fontsize=12)
    ax5.set_xticks(np.linspace(2,12, 6))
    ax5.semilogy()
    ax5.axis([0, 14, 0.001, 10000])
    ax5.set_box_aspect(1)
    
handles,labels = ax1.get_legend_handles_labels()
#ax1.legend( (handles[0], handles[2], handles[4], handles[6], handles[8], handles[10], Line2D([0], [0], color='tab:cyan'), Line2D([0], [0], color='tab:pink'),  Line2D([0], [0], color='green', ls= '--'), Line2D([0], [0], color='green', ls= 'dotted'), Line2D([0], [0], color='k'), Line2D([0], [0], ls='--', color='k'), Line2D([0], [0], ls='dotted', color='k')), ['Kalmag, 71p', 'Kalmag, 50p', 'Kalmag, 0p', 'Kalmag, Neutral_top1', 'Kalmag, Stable_top1', 'Kalmag, S1$^{\gamma}$', 'CovObs.x2, Neutral_top1', 'CHAOS-7.16, Neutral_top1', 'Kalmag, $L_{SV}=13$', 'Kalmag, $L_{SV}=18$', 'SV', 'ER', 'Difference from SV model'], loc='center left', bbox_to_anchor=(1, 0.5))
ax1.grid()
ax3.grid()
ax5.grid()
#plt.show()


lmax = 15
x_U = np.array(np.linspace(1,lmax, lmax))
#fig_U, (ax1) = plt.subplots(1, 1, layout='constrained', sharey=True, figsize=(7,5))
U_spec = []

for i in range(len(Model_list)): 
    model = Model_list[i]
    loc = np.where(PlotModels[model]['Time'] == 2020.0)[0]
    array2D_U = PlotModels[model]['U'][loc][0]
    U_spec = computeUSpectra(lmax, array2D_U)
    ax2.plot(x_U, np.array(U_spec), label = model, color = PlotModels[model]['Color'], ls = '-', marker = PlotModels[model]['Marker'],  markersize=5, lw=1)
    ax2.set_title('U in January 2020', fontsize=14)
    #ax2.set_xlabel('Spherical Harmonic Degree', fontsize=12)
    ax2.set_ylabel('Spectral Energy $(km/yr)^2$', fontsize=12)
    #ax2.set_xticks(np.linspace(2,16, 8))
    ax2.semilogy()
    ax2.axis([0, lmax+1, 0.1, 100])
    ax2.set_box_aspect(1)
    
    loc = np.where(PlotModels[model]['Time'] == 2025.0)[0]
    array2D_U = PlotModels[model]['U'][loc][0]
    U_spec = computeUSpectra(lmax, array2D_U)
    ax4.plot(x_U, np.array(U_spec), label = model, color = PlotModels[model]['Color'], ls = '-', marker = PlotModels[model]['Marker'],  markersize=5, lw=1)
    ax4.set_title('U in January 2025', fontsize=14)
    #ax4.set_xlabel('Spherical Harmonic Degree', fontsize=12)
    ax4.set_ylabel('Spectral Energy $(km/yr)^2$', fontsize=12)
    #ax4.set_xticks(np.linspace(2,16, 8))
    ax4.semilogy()
    ax4.axis([0, lmax+1, 0.1, 100])
    ax4.set_box_aspect(1)
    
    loc = np.where(PlotModels[model]['Time'] == 2030.0)[0]
    array2D_U = PlotModels[model]['U'][loc][0]
    U_spec = computeUSpectra(lmax, array2D_U)
    ax6.plot(x_U, np.array(U_spec), label = model, color = PlotModels[model]['Color'], ls = '-', marker = PlotModels[model]['Marker'],  markersize=5, lw=1)
    ax6.set_title('U in January 2030', fontsize=14)
    ax6.set_xlabel('Spherical Harmonic Degree', fontsize=12)
    ax6.set_ylabel('Spectral Energy $(km/yr)^2$', fontsize=12)
    ax6.set_xticks(np.linspace(2,16, 8))
    ax6.semilogy()
    ax6.axis([0, lmax+1, 0.1, 100])
    ax6.set_box_aspect(1)
    
#ax1.legend( (handles[0], handles[2], handles[4], handles[6], handles[8], handles[10], Line2D([0], [0], color='tab:cyan'), Line2D([0], [0], color='tab:pink'),  Line2D([0], [0], color='green', ls= '--'), Line2D([0], [0], color='green', ls= 'dotted'), Line2D([0], [0], color='k'), Line2D([0], [0], ls='--', color='k'), Line2D([0], [0], ls='dotted', color='k')), ['Kalmag, 71p', 'Kalmag, 50p', 'Kalmag, 0p', 'Kalmag, Neutral_top1', 'Kalmag, Stable_top1', 'Kalmag, S1$^{\gamma}$', 'CovObs.x2, Neutral_top1', 'CHAOS-7.16, Neutral_top1', 'Kalmag, $L_{SV}=13$', 'Kalmag, $L_{SV}=18$', 'SV', 'ER', 'Difference from SV model'], loc='center left', bbox_to_anchor=(1, 0.5))
ax2.grid()
ax4.grid()
ax6.grid()

handles,labels = ax2.get_legend_handles_labels()
ax4.legend(handles, labels, bbox_to_anchor=(1,0.5), loc='center left', fontsize=10.5)
plt.show()


Figure 3: Flow in 2030 for the published Huber weighted mean model in space and the difference between each candidate and the published IGRF model. By altering the timestep, you can also plot the figures in the appendix. You may need to run the PlotModels cell above for this to run

In [ ]:
time_step= 2030.0
loc = np.where(PlotModels['BGS']['Time'] == time_step)[0][0]
Nph = 180
Nth = 91
x = np.linspace(-np.pi,np.pi,Nph)
y = np.linspace(-np.pi/2+0.01,np.pi/2-0.01,Nth)
X,Y = np.meshgrid(x,y)
myphi = x
mytheta = np.pi/2-y
phi1= x*180/np.pi
lat = y*180/np.pi

#####
myfontsize=20
vm=10
dv=0.1
Nlev = int((2*vm/dv)+1)
levels = np.linspace(-vm,vm,Nlev)

sum_diff_map = np.zeros((Nth, Nph))

my_epoch = loc
fig = plt.figure(figsize=(12,11))

## REFERENCE HUBER
PlotModels1 = webgeodyn.models.Models()
ref_model= readPlotmodelhdf5(str(pygeodyn_results)+'Huber_Current_computation.hdf5')
measure = ref_model.measures['U']
components=["th","ph"]
griddata = measure.computeRThetaPhiData(rC, mytheta, myphi,components=components,computeallrealisation=False,irealisation=-1)
ref_Uphi = griddata["ph"]
ref_Uth = griddata["th"]
ref_dated_Uphi = ref_Uphi[my_epoch,:,:]
ref_dated_Uth =  (-1)*ref_Uth[my_epoch,:,:]
U_abs = np.sqrt(ref_dated_Uphi**2+ ref_dated_Uth**2)
PHI, THETA = np.mgrid[-180:180:180j, -90:90:91j]
lw = 5*U_abs / (0.5*np.max(np.sqrt(ref_dated_Uphi**2+ ref_dated_Uth**2)))

levels = np.linspace(-20,20,41)
ax1 = fig.add_subplot(5,3,1,projection=ccrs.Mollweide())
ax1_ref = ax1.contourf(phi1, lat, ref_dated_Uphi, levels, transform=ccrs.PlateCarree(),cmap='RdBu_r',extend='both')
ax1.coastlines(resolution='110m')
ax1.streamplot(phi1, lat, ref_dated_Uphi, ref_dated_Uth,  transform=ccrs.PlateCarree(), density=[1.5, 0.75], color='k', linewidth = lw)
ax1.gridlines()
ax1.set_title(r'HUBER in '+str(time_step))
#plt.colorbar(ax_ref)#, ticks=np.linspace(-10, 10, 5))
#plt.show()

## BGS
PlotModels1 = webgeodyn.models.Models()
my_model= readPlotmodelhdf5(str(pygeodyn_results)+'BGS_Current_computation.hdf5')
measure = my_model.measures['U']
components=["th","ph"]
griddata = measure.computeRThetaPhiData(rC, mytheta, myphi,components=components,computeallrealisation=False,irealisation=-1)
my_Uphi = griddata["ph"]
my_Uth = griddata["th"]
my_dated_Uphi = my_Uphi[my_epoch,:,:] - ref_dated_Uphi
my_dated_Uth =  (-1)*my_Uth[my_epoch,:,:] - ref_dated_Uth
U_abs = np.sqrt(my_dated_Uphi**2+ my_dated_Uth**2)
PHI, THETA = np.mgrid[-180:180:180j, -90:90:91j]
lw = 5*U_abs / (0.5*np.max(np.sqrt(ref_Uphi**2+ ref_Uth**2)))

#fig=plt.figure(figsize=(6,3))
levels = np.linspace(-5,5,21)
ax2 = fig.add_subplot(5,3,2,projection=ccrs.Mollweide())
ax2_ = ax2.contourf(phi1, lat, my_dated_Uphi, levels, transform=ccrs.PlateCarree(),cmap='RdBu_r',extend='both')
ax2.coastlines(resolution='110m')
ax2.streamplot(phi1, lat, my_dated_Uphi, my_dated_Uth,  transform=ccrs.PlateCarree(),  density=[1.5, 0.75], color='k', linewidth = lw)
ax2.gridlines()
corr = np.corrcoef(my_Uphi[my_epoch,:,:].flatten(), ref_Uphi[my_epoch,:,:].flatten())[0,1]
ax2.set_title(r'BGS ('+str(np.round(corr,3))+')')
#plt.colorbar(ax_)#, ticks=np.linspace(-10, 10, 5))
#plt.show()

sum_diff_map = sum_diff_map+U_abs
model1=U_abs
model1_phi= my_dated_Uphi
model1_th = my_dated_Uth

## CSES
PlotModels1 = webgeodyn.models.Models()
my_model= readPlotmodelhdf5(str(pygeodyn_results)+'CSES_Current_computation.hdf5')
measure = my_model.measures['U']
components=["th","ph"]
griddata = measure.computeRThetaPhiData(rC, mytheta, myphi,components=components,computeallrealisation=False,irealisation=-1)
my_Uphi = griddata["ph"]
my_Uth = griddata["th"]
my_dated_Uphi = my_Uphi[my_epoch,:,:] - ref_dated_Uphi
my_dated_Uth =  (-1)*my_Uth[my_epoch,:,:] - ref_dated_Uth
U_abs = np.sqrt(my_dated_Uphi**2+ my_dated_Uth**2)
PHI, THETA = np.mgrid[-180:180:180j, -90:90:91j]
lw = 5*U_abs / (0.5*np.max(np.sqrt(ref_Uphi**2+ ref_Uth**2)))

#fig=plt.figure(figsize=(6,3))
levels = np.linspace(-5,5,21)
ax3 = fig.add_subplot(5,3,3,projection=ccrs.Mollweide())
ax3_ = ax3.contourf(phi1, lat, my_dated_Uphi, levels, transform=ccrs.PlateCarree(),cmap='RdBu_r',extend='both')
ax3.coastlines(resolution='110m')
ax3.streamplot(phi1, lat, my_dated_Uphi, my_dated_Uth,  transform=ccrs.PlateCarree(), density=[1.5, 0.75], color='k', linewidth = lw)
ax3.gridlines()
corr = np.corrcoef(my_Uphi[my_epoch,:,:].flatten(), ref_Uphi[my_epoch,:,:].flatten())[0,1]
ax3.set_title(r'CSES ('+str(np.round(corr,3))+')')

model2=U_abs
model2_phi= my_dated_Uphi
model2_th = my_dated_Uth
sum_diff_map = sum_diff_map+U_abs

#plt.colorbar(ax_)#, ticks=np.linspace(-10, 10, 5))
#plt.show()

## DTU
PlotModels1 = webgeodyn.models.Models()
my_model= readPlotmodelhdf5(str(pygeodyn_results)+'DTU_Current_computation.hdf5')
measure = my_model.measures['U']
components=["th","ph"]
griddata = measure.computeRThetaPhiData(rC, mytheta, myphi,components=components,computeallrealisation=False,irealisation=-1)
my_Uphi = griddata["ph"]
my_Uth = griddata["th"]
my_dated_Uphi = my_Uphi[my_epoch,:,:] - ref_dated_Uphi
my_dated_Uth =  (-1)*my_Uth[my_epoch,:,:] - ref_dated_Uth
U_abs = np.sqrt(my_dated_Uphi**2+ my_dated_Uth**2)
PHI, THETA = np.mgrid[-180:180:180j, -90:90:91j]
lw = 5*U_abs / (0.5*np.max(np.sqrt(ref_Uphi**2+ ref_Uth**2)))

#fig=plt.figure(figsize=(6,3))
levels = np.linspace(-5,5,21)
ax4 = fig.add_subplot(5,3,4,projection=ccrs.Mollweide())
ax4_ = ax4.contourf(phi1, lat, my_dated_Uphi, levels, transform=ccrs.PlateCarree(),cmap='RdBu_r',extend='both')
ax4.coastlines(resolution='110m')
ax4.streamplot(phi1, lat, my_dated_Uphi, my_dated_Uth,  transform=ccrs.PlateCarree(), density=[1.5, 0.75], color='k', linewidth = lw)
ax4.gridlines()
corr = np.corrcoef(my_Uphi[my_epoch,:,:].flatten(), ref_Uphi[my_epoch,:,:].flatten())[0,1]
ax4.set_title(r'DTU ('+str(np.round(corr,3))+')')

model3=U_abs
model3_phi= my_dated_Uphi
model3_th = my_dated_Uth
sum_diff_map = sum_diff_map+U_abs

#plt.colorbar(ax_)#, ticks=np.linspace(-10, 10, 5))
#plt.show()

## GFZ
PlotModels1 = webgeodyn.models.Models()
my_model= readPlotmodelhdf5(str(pygeodyn_results)+'GFZ_Current_computation.hdf5')
measure = my_model.measures['U']
components=["th","ph"]
griddata = measure.computeRThetaPhiData(rC, mytheta, myphi,components=components,computeallrealisation=False,irealisation=-1)
my_Uphi = griddata["ph"]
my_Uth = griddata["th"]
my_dated_Uphi = my_Uphi[my_epoch,:,:] - ref_dated_Uphi
my_dated_Uth =  (-1)*my_Uth[my_epoch,:,:] - ref_dated_Uth
U_abs = np.sqrt(my_dated_Uphi**2+ my_dated_Uth**2)
PHI, THETA = np.mgrid[-180:180:180j, -90:90:91j]
lw = 5*U_abs / (0.5*np.max(np.sqrt(ref_Uphi**2+ ref_Uth**2)))

#fig=plt.figure(figsize=(6,3))
levels = np.linspace(-5,5,21)
ax5 = fig.add_subplot(5,3,5,projection=ccrs.Mollweide())
ax5_ = ax5.contourf(phi1, lat, my_dated_Uphi, levels, transform=ccrs.PlateCarree(),cmap='RdBu_r',extend='both')
ax5.coastlines(resolution='110m')
ax5.streamplot(phi1, lat, my_dated_Uphi, my_dated_Uth,  transform=ccrs.PlateCarree(),  density=[1.5, 0.75], color='k', linewidth = lw)
ax5.gridlines()
corr = np.corrcoef(my_Uphi[my_epoch,:,:].flatten(), ref_Uphi[my_epoch,:,:].flatten())[0,1]
ax5.set_title(r'GFZ ('+str(np.round(corr,3))+')')

model4=U_abs
model4_phi= my_dated_Uphi
model4_th = my_dated_Uth
sum_diff_map = sum_diff_map+U_abs

#plt.colorbar(ax_)#, ticks=np.linspace(-10, 10, 5))
#plt.show()



## IPGP
PlotModels1 = webgeodyn.models.Models()
my_model= readPlotmodelhdf5(str(pygeodyn_results)+'IPGP_Current_computation.hdf5')
measure = my_model.measures['U']
components=["th","ph"]
griddata = measure.computeRThetaPhiData(rC, mytheta, myphi,components=components,computeallrealisation=False,irealisation=-1)
my_Uphi = griddata["ph"]
my_Uth = griddata["th"]
my_dated_Uphi = my_Uphi[my_epoch,:,:] - ref_dated_Uphi
my_dated_Uth =  (-1)*my_Uth[my_epoch,:,:] - ref_dated_Uth
U_abs = np.sqrt(my_dated_Uphi**2+ my_dated_Uth**2)
PHI, THETA = np.mgrid[-180:180:180j, -90:90:91j]
lw = 5*U_abs / (0.5*np.max(np.sqrt(ref_Uphi**2+ ref_Uth**2)))

#fig=plt.figure(figsize=(6,3))
levels = np.linspace(-5,5,21)
ax6 = fig.add_subplot(5,3,6,projection=ccrs.Mollweide())
ax6_ = ax6.contourf(phi1, lat, my_dated_Uphi, levels, transform=ccrs.PlateCarree(),cmap='RdBu_r',extend='both')
ax6.coastlines(resolution='110m')
ax6.streamplot(phi1, lat, my_dated_Uphi, my_dated_Uth,  transform=ccrs.PlateCarree(),  density=[1.5, 0.75], color='k', linewidth = lw)
ax6.gridlines()
corr = np.corrcoef(my_Uphi[my_epoch,:,:].flatten(), ref_Uphi[my_epoch,:,:].flatten())[0,1]
ax6.set_title(r'IPGP ('+str(np.round(corr,3))+')')

model5=U_abs
model5_phi= my_dated_Uphi
model5_th = my_dated_Uth
sum_diff_map = sum_diff_map+U_abs

#plt.colorbar(ax_)#, ticks=np.linspace(-10, 10, 5))
#plt.show()

## ISTERRE
PlotModels1 = webgeodyn.models.Models()
my_model= readPlotmodelhdf5(str(pygeodyn_results)+'ISTERRE_Current_computation.hdf5')
measure = my_model.measures['U']
components=["th","ph"]
griddata = measure.computeRThetaPhiData(rC, mytheta, myphi,components=components,computeallrealisation=False,irealisation=-1)
my_Uphi = griddata["ph"]
my_Uth = griddata["th"]
my_dated_Uphi = my_Uphi[my_epoch,:,:] - ref_dated_Uphi
my_dated_Uth =  (-1)*my_Uth[my_epoch,:,:] - ref_dated_Uth
U_abs = np.sqrt(my_dated_Uphi**2+ my_dated_Uth**2)
PHI, THETA = np.mgrid[-180:180:180j, -90:90:91j]
lw = 5*U_abs / (0.5*np.max(np.sqrt(ref_Uphi**2+ ref_Uth**2)))

#fig=plt.figure(figsize=(6,3))
levels = np.linspace(-5,5,21)
ax7 = fig.add_subplot(5,3,7,projection=ccrs.Mollweide())
ax7_ = ax7.contourf(phi1, lat, my_dated_Uphi, levels, transform=ccrs.PlateCarree(),cmap='RdBu_r',extend='both')
ax7.coastlines(resolution='110m')
ax7.streamplot(phi1, lat, my_dated_Uphi, my_dated_Uth,  transform=ccrs.PlateCarree(), density=[1.5, 0.75], color='k', linewidth = lw)
ax7.gridlines()
corr = np.corrcoef(my_Uphi[my_epoch,:,:].flatten(), ref_Uphi[my_epoch,:,:].flatten())[0,1]
ax7.set_title(r'ISTERRE ('+str(np.round(corr,3))+')')

model6=U_abs
model6_phi= my_dated_Uphi
model6_th = my_dated_Uth
sum_diff_map = sum_diff_map+U_abs

#plt.colorbar(ax_)#, ticks=np.linspace(-10, 10, 5))
#plt.show()


## MISTA
PlotModels1 = webgeodyn.models.Models()
my_model= readPlotmodelhdf5(str(pygeodyn_results)+'MISTA_Current_computation.hdf5')
measure = my_model.measures['U']
components=["th","ph"]
griddata = measure.computeRThetaPhiData(rC, mytheta, myphi,components=components,computeallrealisation=False,irealisation=-1)
my_Uphi = griddata["ph"]
my_Uth = griddata["th"]
my_dated_Uphi = my_Uphi[my_epoch,:,:] - ref_dated_Uphi
my_dated_Uth =  (-1)*my_Uth[my_epoch,:,:] - ref_dated_Uth
U_abs = np.sqrt(my_dated_Uphi**2+ my_dated_Uth**2)
PHI, THETA = np.mgrid[-180:180:180j, -90:90:91j]
lw = 5*U_abs / (0.5*np.max(np.sqrt(ref_Uphi**2+ ref_Uth**2)))

#fig=plt.figure(figsize=(6,3))
levels = np.linspace(-5,5,21)
ax9 = fig.add_subplot(5,3,9,projection=ccrs.Mollweide())
ax9_ = ax9.contourf(phi1, lat, my_dated_Uphi, levels, transform=ccrs.PlateCarree(),cmap='RdBu_r',extend='both')
ax9.coastlines(resolution='110m')
ax9.streamplot(phi1, lat, my_dated_Uphi, my_dated_Uth,  transform=ccrs.PlateCarree(),  density=[1.5, 0.75], color='k', linewidth = lw)
ax9.gridlines()
corr = np.corrcoef(my_Uphi[my_epoch,:,:].flatten(), ref_Uphi[my_epoch,:,:].flatten())[0,1]
ax9.set_title(r'MISTA ('+str(np.round(corr,3))+')')

model7=U_abs
model7_phi= my_dated_Uphi
model7_th = my_dated_Uth
sum_diff_map = sum_diff_map+U_abs

#plt.colorbar(ax_)
#plt.show()

## NOAA
PlotModels1 = webgeodyn.models.Models()
my_model= readPlotmodelhdf5(str(pygeodyn_results)+'NOAA_Current_computation.hdf5')
measure = my_model.measures['U']
components=["th","ph"]
griddata = measure.computeRThetaPhiData(rC, mytheta, myphi,components=components,computeallrealisation=False,irealisation=-1)
my_Uphi = griddata["ph"]
my_Uth = griddata["th"]
my_dated_Uphi = my_Uphi[my_epoch,:,:] - ref_dated_Uphi
my_dated_Uth =  (-1)*my_Uth[my_epoch,:,:] - ref_dated_Uth
U_abs = np.sqrt(my_dated_Uphi**2+ my_dated_Uth**2)
PHI, THETA = np.mgrid[-180:180:180j, -90:90:91j]
lw = 5*U_abs / (0.5*np.max(np.sqrt(ref_Uphi**2+ ref_Uth**2)))

#fig=plt.figure(figsize=(6,3))
levels = np.linspace(-5,5,21)
ax10 = fig.add_subplot(5,3,10,projection=ccrs.Mollweide())
ax10_ = ax10.contourf(phi1, lat, my_dated_Uphi, levels, transform=ccrs.PlateCarree(),cmap='RdBu_r',extend='both')
ax10.coastlines(resolution='110m')
ax10.streamplot(phi1, lat, my_dated_Uphi, my_dated_Uth,  transform=ccrs.PlateCarree(),  density=[1.5, 0.75], color='k', linewidth = lw)
ax10.gridlines()
corr = np.corrcoef(my_Uphi[my_epoch,:,:].flatten(), ref_Uphi[my_epoch,:,:].flatten())[0,1]
ax10.set_title(r'NOAA ('+str(np.round(corr,3))+')')

model8=U_abs
model8_phi= my_dated_Uphi
model8_th = my_dated_Uth
sum_diff_map = sum_diff_map+U_abs

#plt.colorbar(ax_)
#plt.show()

## Strasbourg
PlotModels1 = webgeodyn.models.Models()
my_model= readPlotmodelhdf5(str(pygeodyn_results)+'Strasbourg_Current_computation.hdf5')
measure = my_model.measures['U']
components=["th","ph"]
griddata = measure.computeRThetaPhiData(rC, mytheta, myphi,components=components,computeallrealisation=False,irealisation=-1)
my_Uphi = griddata["ph"]
my_Uth = griddata["th"]
my_dated_Uphi = my_Uphi[my_epoch,:,:] - ref_dated_Uphi
my_dated_Uth =  (-1)*my_Uth[my_epoch,:,:] - ref_dated_Uth
U_abs = np.sqrt(my_dated_Uphi**2+ my_dated_Uth**2)
PHI, THETA = np.mgrid[-180:180:180j, -90:90:91j]
lw = 5*U_abs / (0.5*np.max(np.sqrt(ref_Uphi**2+ ref_Uth**2)))

#fig=plt.figure(figsize=(6,3))
levels = np.linspace(-5,5,21)
ax8 = fig.add_subplot(5,3,8,projection=ccrs.Mollweide())
ax8_ = ax8.contourf(phi1, lat, my_dated_Uphi, levels, transform=ccrs.PlateCarree(),cmap='RdBu_r',extend='both')
ax8.coastlines(resolution='110m')
ax8.streamplot(phi1, lat, my_dated_Uphi, my_dated_Uth,  transform=ccrs.PlateCarree(),  density=[1.5, 0.75], color='k', linewidth = lw)
ax8.gridlines()
corr = np.corrcoef(my_Uphi[my_epoch,:,:].flatten(), ref_Uphi[my_epoch,:,:].flatten())[0,1]
ax8.set_title(r'ITES ('+str(np.round(corr,3))+')')

model9=U_abs
model9_phi= my_dated_Uphi
model9_th = my_dated_Uth
sum_diff_map = sum_diff_map+U_abs

#plt.colorbar(ax_)
#plt.show()

## TU_Berlin
PlotModels1 = webgeodyn.models.Models()
my_model= readPlotmodelhdf5(str(pygeodyn_results)+'TU_Berlin_Current_computation.hdf5')
measure = my_model.measures['U']
components=["th","ph"]
griddata = measure.computeRThetaPhiData(rC, mytheta, myphi,components=components,computeallrealisation=False,irealisation=-1)
my_Uphi = griddata["ph"]
my_Uth = griddata["th"]
my_dated_Uphi = my_Uphi[my_epoch,:,:] - ref_dated_Uphi
my_dated_Uth =  (-1)*my_Uth[my_epoch,:,:] - ref_dated_Uth
U_abs = np.sqrt(my_dated_Uphi**2+ my_dated_Uth**2)
PHI, THETA = np.mgrid[-180:180:180j, -90:90:91j]
lw = 5*U_abs / (0.5*np.max(np.sqrt(ref_Uphi**2+ ref_Uth**2)))

#fig=plt.figure(figsize=(6,3))
levels = np.linspace(-5,5,21)
ax11 = fig.add_subplot(5,3,11,projection=ccrs.Mollweide())
ax11.coastlines(resolution='110m')
ax11.streamplot(phi1, lat, my_dated_Uphi, my_dated_Uth,  transform=ccrs.PlateCarree(),  density=[1.5, 0.75], color='k', linewidth = lw)
ax11.gridlines()
corr = np.corrcoef(my_Uphi[my_epoch,:,:].flatten(), ref_Uphi[my_epoch,:,:].flatten())[0,1]
ax11.set_title(r'TUB ('+str(np.round(corr,3))+')')

model10=U_abs
model10_phi= my_dated_Uphi
model10_th = my_dated_Uth
sum_diff_map = sum_diff_map+U_abs

#plt.colorbar(ax_)
#plt.show()

## UCM
PlotModels1 = webgeodyn.models.Models()
my_model= readPlotmodelhdf5(str(pygeodyn_results)+'UCM_Current_computation.hdf5')
measure = my_model.measures['U']
components=["th","ph"]
griddata = measure.computeRThetaPhiData(rC, mytheta, myphi,components=components,computeallrealisation=False,irealisation=-1)
my_Uphi = griddata["ph"]
my_Uth = griddata["th"]
my_dated_Uphi = my_Uphi[my_epoch,:,:] - ref_dated_Uphi
my_dated_Uth =  (-1)*my_Uth[my_epoch,:,:] - ref_dated_Uth
U_abs = np.sqrt(my_dated_Uphi**2+ my_dated_Uth**2)
PHI, THETA = np.mgrid[-180:180:180j, -90:90:91j]
lw = 5*U_abs / (0.5*np.max(np.sqrt(ref_Uphi**2+ ref_Uth**2)))

#fig=plt.figure(figsize=(6,3))
levels = np.linspace(-5,5,21)
ax12 = fig.add_subplot(5,3,12,projection=ccrs.Mollweide())
ax12_ = ax12.contourf(phi1, lat, my_dated_Uphi, levels, transform=ccrs.PlateCarree(),cmap='RdBu_r',extend='both')
ax12.coastlines(resolution='110m')
ax12.streamplot(phi1, lat, my_dated_Uphi, my_dated_Uth,  transform=ccrs.PlateCarree(),  density=[1.5, 0.75], color='k', linewidth = lw)
ax12.gridlines()
corr = np.corrcoef(my_Uphi[my_epoch,:,:].flatten(), ref_Uphi[my_epoch,:,:].flatten())[0,1]
ax12.set_title(r'UCM ('+str(np.round(corr,3))+')')


model11=U_abs
model11_phi= my_dated_Uphi
model11_th = my_dated_Uth
sum_diff_map = sum_diff_map+U_abs

#plt.colorbar(ax_)
#plt.show()

## USTHB
PlotModels1 = webgeodyn.models.Models()
my_model= readPlotmodelhdf5(str(pygeodyn_results)+'USTHB_Current_computation.hdf5')
measure = my_model.measures['U']
components=["th","ph"]
griddata = measure.computeRThetaPhiData(rC, mytheta, myphi,components=components,computeallrealisation=False,irealisation=-1)
my_Uphi = griddata["ph"]
my_Uth = griddata["th"]
my_dated_Uphi = my_Uphi[my_epoch,:,:] - ref_dated_Uphi
my_dated_Uth =  (-1)*my_Uth[my_epoch,:,:] - ref_dated_Uth
U_abs = np.sqrt(my_dated_Uphi**2+ my_dated_Uth**2)
PHI, THETA = np.mgrid[-180:180:180j, -90:90:91j]
lw = 5*U_abs / (0.5*np.max(np.sqrt(ref_Uphi**2+ ref_Uth**2)))

#fig=plt.figure(figsize=(6,3))
levels = np.linspace(-5,5,21)
ax13 = fig.add_subplot(5,3,13,projection=ccrs.Mollweide())
ax13_ = ax13.contourf(phi1, lat, my_dated_Uphi, levels, transform=ccrs.PlateCarree(),cmap='RdBu_r',extend='both')
ax13.coastlines(resolution='110m')
ax13.streamplot(phi1, lat, my_dated_Uphi, my_dated_Uth,  transform=ccrs.PlateCarree(),  density=[1.5, 0.75], color='k', linewidth = lw)
ax13.gridlines()
corr = np.corrcoef(my_Uphi[my_epoch,:,:].flatten(), ref_Uphi[my_epoch,:,:].flatten())[0,1]
ax13.set_title(r'USTHB ('+str(np.round(corr,3))+')')
model12=U_abs
model12_phi= my_dated_Uphi
model12_th = my_dated_Uth
sum_diff_map = sum_diff_map+U_abs

#plt.colorbar(ax_)
#plt.show()

## WHU
PlotModels1 = webgeodyn.models.Models()
my_model= readPlotmodelhdf5(str(pygeodyn_results)+'WHU_Current_computation.hdf5')
measure = my_model.measures['U']
components=["th","ph"]
griddata = measure.computeRThetaPhiData(rC, mytheta, myphi,components=components,computeallrealisation=False,irealisation=-1)
my_Uphi = griddata["ph"]
my_Uth = griddata["th"]
my_dated_Uphi = my_Uphi[my_epoch,:,:] - ref_dated_Uphi
my_dated_Uth =  (-1)*my_Uth[my_epoch,:,:] - ref_dated_Uth
U_abs = np.sqrt(my_dated_Uphi**2+ my_dated_Uth**2)
PHI, THETA = np.mgrid[-180:180:180j, -90:90:91j]
lw = 5*U_abs / (0.5*np.max(np.sqrt(ref_Uphi**2+ ref_Uth**2)))

#fig=plt.figure(figsize=(6,3))
levels = np.linspace(-5,5,21)
ax14 = fig.add_subplot(5,3,14,projection=ccrs.Mollweide())
ax14_ = ax14.contourf(phi1, lat, my_dated_Uphi, levels, transform=ccrs.PlateCarree(),cmap='RdBu_r',extend='both')
ax14.coastlines(resolution='110m')
ax14.streamplot(phi1, lat, my_dated_Uphi, my_dated_Uth,  transform=ccrs.PlateCarree(),  density=[1.5, 0.75], color='k', linewidth = lw)
ax14.gridlines()
corr = np.corrcoef(my_Uphi[my_epoch,:,:].flatten(), ref_Uphi[my_epoch,:,:].flatten())[0,1]
ax14.set_title(r'WHU ('+str(np.round(corr,3))+')')

model13=U_abs
model13_phi= my_dated_Uphi
model13_th = my_dated_Uth
sum_diff_map = sum_diff_map+U_abs

#plt.colorbar(ax_)

plt.show()


Figure 4: Spatial Huber weighed mean of deviation of flow inversion (see Fig 3). You will need to run the cell above for this to run

In [ ]:
maps = np.stack([model1, model2, model3, model4, model5, model6, model7,model8, model9, model10, model11, model12, model13])
maps_phi = np.stack([model1_phi, model2_phi, model3_phi, model4_phi, model5_phi, model6_phi, model7_phi,model8_phi, model9_phi, model10_phi, model11_phi, model12_phi, model13_phi])
maps_th = np.stack([model1_th, model2_th, model3_th, model4_th, model5_th, model6_th, model7_th,model8_th, model9_th, model10_th, model11_th, model12_th, model13_th])

huber_mean_ph, huber_wts_ph = compute_huber_weighted_mean(maps_phi)
huber_mean_th, huber_wts_th = compute_huber_weighted_mean(maps_th)
huber_mean, huber_wts = compute_huber_weighted_mean(maps)


In [ ]:
# Broadcast mean to match maps shape
mean_broadcast_ph = np.broadcast_to(huber_mean_ph, maps.shape)
mean_broadcast_th = np.broadcast_to(huber_mean_th, maps.shape)
mean_broadcast = np.broadcast_to(huber_mean, maps.shape)

# Absolute deviation from the Huber mean
abs_dev_ph = np.abs(maps - mean_broadcast_ph)  # shape: (num_models, H, W)
abs_dev_th = np.abs(maps - mean_broadcast_th)  # shape: (num_models, H, W)
abs_dev = np.abs(maps - mean_broadcast)  # shape: (num_models, H, W)

# Weighted deviation
weighted_dev_ph = abs_dev_ph * huber_wts_ph  # shape: (num_models, H, W)
weighted_dev_th = abs_dev_th * huber_wts_th  # shape: (num_models, H, W)
weighted_dev = abs_dev * huber_wts  # shape: (num_models, H, W)

# --- Option 1: Mean deviation per pixel (across models) ---
spatial_deviation_map_ph = np.nanmean(weighted_dev_ph, axis=0)  # shape: (H, W)
spatial_deviation_map_th = np.nanmean(weighted_dev_th, axis=0)  # shape: (H, W)
spatial_deviation_map = np.nanmean(weighted_dev, axis=0)  # shape: (H, W)

# --- Option 2: Max deviation per pixel ---
# spatial_deviation_map = np.nanmax(weighted_dev, axis=0)

#plt.figure(figsize=(10, 5))
#plt.imshow(spatial_deviation_map, cmap='Reds')
#plt.colorbar(label='Mean Weighted Deviation')
#plt.xlabel('Longitude Index')
#plt.ylabel('Latitude Index')
#plt.tight_layout()
#plt.show()

print(r'Minimum $U_\phi$ huber weighted deviation:'+str(np.min(spatial_deviation_map_ph))+r', Maximum $U_\phi$ huber weighted deviation:'+str(np.max(spatial_deviation_map_ph)))
fig = plt.figure(figsize=(8,5))
levels = np.linspace(0,2.4,49)
ax1 = fig.add_subplot(1,1,1,projection=ccrs.Mollweide())
ax1_ = ax1.contourf(phi1, lat, spatial_deviation_map_ph, levels, transform=ccrs.PlateCarree(),cmap='Reds',extend='both')
ax1.coastlines(resolution='110m')
ax1.gridlines()
plt.title(r'Spatial map of weighted $U_\phi$ flow speed deviations from Huber mean')
plt.colorbar(ax1_, label=r'Flow speed deviation (km/yr)', cmap='RdBu_r', location = 'bottom')
plt.show()

print(r'Minimum $U_\theta$ huber weighted deviation:'+str(np.min(spatial_deviation_map_th))+r', Maximum $U_\theta$ huber weighted deviation:'+str(np.max(spatial_deviation_map_th)))
fig = plt.figure(figsize=(8,5))
levels = np.linspace(0,2.4,49)
ax1 = fig.add_subplot(1,1,1,projection=ccrs.Mollweide())
ax1_ = ax1.contourf(phi1, lat, spatial_deviation_map_th,levels, transform=ccrs.PlateCarree(),cmap='Reds')
ax1.coastlines(resolution='110m')
ax1.gridlines()
plt.title(r'Spatial map of weighted $U_\theta$ flow speed deviations from Huber mean')
plt.colorbar(ax1_, label=r'Flow speed deviation (km/yr)', cmap='RdBu_r', location = 'bottom')
plt.show()

print(r'Minimum $U$ huber weighted deviation:'+str(np.min(spatial_deviation_map))+', Maximum $U$ huber weighted deviation:'+str(np.max(spatial_deviation_map)))
fig = plt.figure(figsize=(8,5))
levels = np.linspace(0,2.4,49)
ax1 = fig.add_subplot(1,1,1,projection=ccrs.Mollweide())
ax1_ = ax1.contourf(phi1, lat, spatial_deviation_map, levels, transform=ccrs.PlateCarree(),cmap='Reds',extend='both')
ax1.coastlines(resolution='110m')
ax1.gridlines()
plt.title(r'Spatial map of weighted absolute flow speed deviations from Huber mean')
plt.colorbar(ax1_, label=r'Flow speed deviation (km/yr)', cmap='RdBu_r', location = 'bottom')
plt.show()

Figure 5: Time-longitude plot of Uphi at the equator over the 10 year period

In [ ]:
TL_start = 2020.0
TL_end = 2030.0
loc_start = np.where(PlotModels['BGS']['Time'] == TL_start)[0][0]
loc_end = np.where(PlotModels['BGS']['Time'] == TL_end)[0][0]
color_speed = 20
lat_request = 91
Nph = 360
Nth = 181
x = np.linspace(0,2*np.pi,Nph)
y = np.linspace(-np.pi/2+0.01,np.pi/2-0.01,Nth)
X,Y = np.meshgrid(x,y)
myphi = x
mytheta = np.pi/2-y
phi1= x*180/np.pi
lat = y*180/np.pi
cmap= 'RdBu_r'

#####
myfontsize=20
vm=20
dv=0.1

fig_TL, ((ax1, ax2, ax3), (ax4, ax5, ax6), (ax7, ax8, ax9), (ax10, ax11, ax12), (ax13, ax14, ax15)) = plt.subplots(5, 3, layout='constrained', sharex=True,sharey=True, figsize=(8,8))
#fig1, ((ax1, ax2), (ax3, ax4), (ax5, ax6), (ax7, ax8)) = plt.subplots(4, 2, layout='constrained', figsize=(8,8), sharex= True, sharey = True)

ref_model= readPlotmodelhdf5(str(pygeodyn_results)+'Huber_Current_computation.hdf5')
measure = ref_model.measures['U']
components=["th","ph"]
griddata = measure.computeRThetaPhiData(rC, mytheta, myphi,components=components,computeallrealisation=False,irealisation=-1)
ref_Uphi = griddata["ph"]

ref_eq_Uphi = np.zeros((loc_end-loc_start, Nph))
for i in range(Nph):
    ref_eq_Uphi[:,i] = ref_Uphi[loc_start:loc_end,lat_request,i]
ax1.imshow(np.flipud(ref_eq_Uphi), cmap=cmap, extent=[0, 360, TL_start, TL_end], aspect='auto', vmin=-color_speed, vmax=color_speed)
ax1.set_title(r'Huber')


my_model= readPlotmodelhdf5(str(pygeodyn_results)+'BGS_Current_computation.hdf5')
measure = my_model.measures['U']
components=["th","ph"]
griddata = measure.computeRThetaPhiData(rC, mytheta, myphi,components=components,computeallrealisation=False,irealisation=-1)
my_Uphi = griddata["ph"]

my_eq_Uphi = np.zeros((loc_end-loc_start, Nph))
for i in range(Nph):
    my_eq_Uphi[:,i] = my_Uphi[loc_start:loc_end,lat_request,i] #- ref_Uphi[loc_start:loc_end,lat_request,i]
ax2.imshow(np.flipud(my_eq_Uphi),cmap=cmap, extent=[0, 360, TL_start, TL_end], aspect='auto', vmin=-color_speed, vmax=color_speed)

epoch1 = 4
epoch2 = 9
corr2023 = np.corrcoef(my_eq_Uphi[epoch1,:].flatten(), ref_eq_Uphi[epoch1,:].flatten())[0,1]
corr2028 = np.corrcoef(my_eq_Uphi[epoch2:].flatten(), ref_eq_Uphi[epoch2,:].flatten())[0,1]
ax2.set_title(r'BGS ('+str(np.round(corr2023,3))+', '+str(np.round(corr2028,3))+')')

#fig_TL, (ax1) = plt.subplots(1, 1, layout='constrained', sharey=True, figsize=(7,5))
#fig1, ((ax1, ax2), (ax3, ax4), (ax5, ax6), (ax7, ax8)) = plt.subplots(4, 2, layout='constrained', figsize=(8,8), sharex= True, sharey = True)

my_model= readPlotmodelhdf5(str(pygeodyn_results)+'CSES_Current_computation.hdf5')
measure = my_model.measures['U']
components=["th","ph"]
griddata = measure.computeRThetaPhiData(rC, mytheta, myphi,components=components,computeallrealisation=False,irealisation=-1)
my_Uphi = griddata["ph"]

my_eq_Uphi = np.zeros((loc_end-loc_start, Nph))
for i in range(Nph):
    my_eq_Uphi[:,i] = my_Uphi[loc_start:loc_end,lat_request,i]#- ref_Uphi[loc_start:loc_end,lat_request,i]
ax3.imshow(np.flipud(my_eq_Uphi),cmap=cmap, extent=[0, 360, TL_start, TL_end], aspect='auto', vmin=-color_speed, vmax=color_speed)
corr2023 = np.corrcoef(my_eq_Uphi[epoch1,:].flatten(), ref_eq_Uphi[epoch1,:].flatten())[0,1]
corr2028 = np.corrcoef(my_eq_Uphi[epoch2:].flatten(), ref_eq_Uphi[epoch2,:].flatten())[0,1]
ax3.set_title(r'CSES ('+str(np.round(corr2023,3))+', '+str(np.round(corr2028,3))+')')

my_model= readPlotmodelhdf5(str(pygeodyn_results)+'DTU_Current_computation.hdf5')
measure = my_model.measures['U']
components=["th","ph"]
griddata = measure.computeRThetaPhiData(rC, mytheta, myphi,components=components,computeallrealisation=False,irealisation=-1)
my_Uphi = griddata["ph"]

my_eq_Uphi = np.zeros((loc_end-loc_start, Nph))
for i in range(Nph):
    my_eq_Uphi[:,i] = my_Uphi[loc_start:loc_end,lat_request,i]#- ref_Uphi[loc_start:loc_end,lat_request,i]
ax4.imshow(np.flipud(my_eq_Uphi),cmap=cmap, extent=[0, 360, TL_start, TL_end], aspect='auto', vmin=-color_speed, vmax=color_speed)
corr2023 = np.corrcoef(my_eq_Uphi[epoch1,:].flatten(), ref_eq_Uphi[epoch1,:].flatten())[0,1]
corr2028 = np.corrcoef(my_eq_Uphi[epoch2:].flatten(), ref_eq_Uphi[epoch2,:].flatten())[0,1]
ax4.set_title(r'DTU ('+str(np.round(corr2023,3))+', '+str(np.round(corr2028,3))+')')

my_model= readPlotmodelhdf5(str(pygeodyn_results)+'GFZ_Current_computation.hdf5')
measure = my_model.measures['U']
components=["th","ph"]
griddata = measure.computeRThetaPhiData(rC, mytheta, myphi,components=components,computeallrealisation=False,irealisation=-1)
my_Uphi = griddata["ph"]

my_eq_Uphi = np.zeros((loc_end-loc_start, Nph))
for i in range(Nph):
    my_eq_Uphi[:,i] = my_Uphi[loc_start:loc_end,lat_request,i]#- ref_Uphi[loc_start:loc_end,lat_request,i]
ax5.imshow(np.flipud(my_eq_Uphi),cmap=cmap, extent=[0, 360, TL_start, TL_end], aspect='auto', vmin=-color_speed, vmax=color_speed)
corr2023 = np.corrcoef(my_eq_Uphi[epoch1,:].flatten(), ref_eq_Uphi[epoch1,:].flatten())[0,1]
corr2028 = np.corrcoef(my_eq_Uphi[epoch2:].flatten(), ref_eq_Uphi[epoch2,:].flatten())[0,1]
ax5.set_title(r'GFZ ('+str(np.round(corr2023,3))+', '+str(np.round(corr2028,3))+')')

my_model= readPlotmodelhdf5(str(pygeodyn_results)+'IPGP_Current_computation.hdf5')
measure = my_model.measures['U']
components=["th","ph"]
griddata = measure.computeRThetaPhiData(rC, mytheta, myphi,components=components,computeallrealisation=False,irealisation=-1)
my_Uphi = griddata["ph"]

my_eq_Uphi = np.zeros((loc_end-loc_start, Nph))
for i in range(Nph):
    my_eq_Uphi[:,i] = my_Uphi[loc_start:loc_end,lat_request,i]#- ref_Uphi[loc_start:loc_end,lat_request,i]
ax6.imshow(np.flipud(my_eq_Uphi),cmap=cmap, extent=[0, 360, TL_start, TL_end], aspect='auto', vmin=-color_speed, vmax=color_speed)
corr2023 = np.corrcoef(my_eq_Uphi[epoch1,:].flatten(), ref_eq_Uphi[epoch1,:].flatten())[0,1]
corr2028 = np.corrcoef(my_eq_Uphi[epoch2:].flatten(), ref_eq_Uphi[epoch2,:].flatten())[0,1]
ax6.set_title(r'IPGP ('+str(np.round(corr2023,3))+', '+str(np.round(corr2028,3))+')')

my_model= readPlotmodelhdf5(str(pygeodyn_results)+'ISTERRE_Current_computation.hdf5')
measure = my_model.measures['U']
components=["th","ph"]
griddata = measure.computeRThetaPhiData(rC, mytheta, myphi,components=components,computeallrealisation=False,irealisation=-1)
my_Uphi = griddata["ph"]

my_eq_Uphi = np.zeros((loc_end-loc_start, Nph))
for i in range(Nph):
    my_eq_Uphi[:,i] = my_Uphi[loc_start:loc_end,lat_request,i]#- ref_Uphi[loc_start:loc_end,lat_request,i]
ax7.imshow(np.flipud(my_eq_Uphi),cmap=cmap, extent=[0, 360, TL_start, TL_end], aspect='auto', vmin=-color_speed, vmax=color_speed)
corr2023 = np.corrcoef(my_eq_Uphi[epoch1,:].flatten(), ref_eq_Uphi[epoch1,:].flatten())[0,1]
corr2028 = np.corrcoef(my_eq_Uphi[epoch2:].flatten(), ref_eq_Uphi[epoch2,:].flatten())[0,1]
ax7.set_title(r'ISTERRE ('+str(np.round(corr2023,3))+', '+str(np.round(corr2028,3))+')')

my_model= readPlotmodelhdf5(str(pygeodyn_results)+'MISTA_Current_computation.hdf5')
measure = my_model.measures['U']
components=["th","ph"]
griddata = measure.computeRThetaPhiData(rC, mytheta, myphi,components=components,computeallrealisation=False,irealisation=-1)
my_Uphi = griddata["ph"]

my_eq_Uphi = np.zeros((loc_end-loc_start, Nph))
for i in range(Nph):
    my_eq_Uphi[:,i] = my_Uphi[loc_start:loc_end,lat_request,i]#- ref_Uphi[loc_start:loc_end,lat_request,i]
ax9.imshow(np.flipud(my_eq_Uphi),cmap=cmap, extent=[0, 360, TL_start, TL_end], aspect='auto', vmin=-color_speed, vmax=color_speed)
corr2023 = np.corrcoef(my_eq_Uphi[epoch1,:].flatten(), ref_eq_Uphi[epoch1,:].flatten())[0,1]
corr2028 = np.corrcoef(my_eq_Uphi[epoch2:].flatten(), ref_eq_Uphi[epoch2,:].flatten())[0,1]
ax9.set_title(r'MISTA ('+str(np.round(corr2023,3))+', '+str(np.round(corr2028,3))+')')

my_model= readPlotmodelhdf5(str(pygeodyn_results)+'NOAA_Current_computation.hdf5')
measure = my_model.measures['U']
components=["th","ph"]
griddata = measure.computeRThetaPhiData(rC, mytheta, myphi,components=components,computeallrealisation=False,irealisation=-1)
my_Uphi = griddata["ph"]

my_eq_Uphi = np.zeros((loc_end-loc_start, Nph))
for i in range(Nph):
    my_eq_Uphi[:,i] = my_Uphi[loc_start:loc_end,lat_request,i]#- ref_Uphi[loc_start:loc_end,lat_request,i]
ax10.imshow(np.flipud(my_eq_Uphi),cmap=cmap, extent=[0, 360, TL_start, TL_end], aspect='auto', vmin=-color_speed, vmax=color_speed)
corr2023 = np.corrcoef(my_eq_Uphi[epoch1,:].flatten(), ref_eq_Uphi[epoch1,:].flatten())[0,1]
corr2028 = np.corrcoef(my_eq_Uphi[epoch2:].flatten(), ref_eq_Uphi[epoch2,:].flatten())[0,1]
ax10.set_title(r'NOAA ('+str(np.round(corr2023,3))+', '+str(np.round(corr2028,3))+')')

my_model= readPlotmodelhdf5(str(pygeodyn_results)+'Strasbourg_Current_computation.hdf5')
measure = my_model.measures['U']
components=["th","ph"]
griddata = measure.computeRThetaPhiData(rC, mytheta, myphi,components=components,computeallrealisation=False,irealisation=-1)
my_Uphi = griddata["ph"]

my_eq_Uphi = np.zeros((loc_end-loc_start, Nph))
for i in range(Nph):
    my_eq_Uphi[:,i] = my_Uphi[loc_start:loc_end,lat_request,i]#- ref_Uphi[loc_start:loc_end,lat_request,i]
ax8.imshow(np.flipud(my_eq_Uphi),cmap=cmap, extent=[0, 360, TL_start, TL_end], aspect='auto', vmin=-color_speed, vmax=color_speed)
corr2023 = np.corrcoef(my_eq_Uphi[epoch1,:].flatten(), ref_eq_Uphi[epoch1,:].flatten())[0,1]
corr2028 = np.corrcoef(my_eq_Uphi[epoch2:].flatten(), ref_eq_Uphi[epoch2,:].flatten())[0,1]
ax8.set_title(r'ITES ('+str(np.round(corr2023,3))+', '+str(np.round(corr2028,3))+')')

my_model= readPlotmodelhdf5(str(pygeodyn_results)+'TU_Berlin_Current_computation.hdf5')
measure = my_model.measures['U']
components=["th","ph"]
griddata = measure.computeRThetaPhiData(rC, mytheta, myphi,components=components,computeallrealisation=False,irealisation=-1)
my_Uphi = griddata["ph"]

my_eq_Uphi = np.zeros((loc_end-loc_start, Nph))
for i in range(Nph):
    my_eq_Uphi[:,i] = my_Uphi[loc_start:loc_end,lat_request,i]#- ref_Uphi[loc_start:loc_end,lat_request,i]
ax11.imshow(np.flipud(my_eq_Uphi),cmap=cmap, extent=[0, 360, TL_start, TL_end], aspect='auto', vmin=-color_speed, vmax=color_speed)
corr2023 = np.corrcoef(my_eq_Uphi[epoch1,:].flatten(), ref_eq_Uphi[epoch1,:].flatten())[0,1]
corr2028 = np.corrcoef(my_eq_Uphi[epoch2:].flatten(), ref_eq_Uphi[epoch2,:].flatten())[0,1]
ax11.set_title(r'TUB ('+str(np.round(corr2023,3))+', '+str(np.round(corr2028,3))+')')

my_model= readPlotmodelhdf5(str(pygeodyn_results)+'UCM_Current_computation.hdf5')
measure = my_model.measures['U']
components=["th","ph"]
griddata = measure.computeRThetaPhiData(rC, mytheta, myphi,components=components,computeallrealisation=False,irealisation=-1)
my_Uphi = griddata["ph"]

my_eq_Uphi = np.zeros((loc_end-loc_start, Nph))
for i in range(Nph):
    my_eq_Uphi[:,i] = my_Uphi[loc_start:loc_end,lat_request,i]#- ref_Uphi[loc_start:loc_end,lat_request,i]
ax12.imshow(np.flipud(my_eq_Uphi),cmap=cmap, extent=[0, 360, TL_start, TL_end], aspect='auto', vmin=-color_speed, vmax=color_speed)
corr2023 = np.corrcoef(my_eq_Uphi[epoch1,:].flatten(), ref_eq_Uphi[epoch1,:].flatten())[0,1]
corr2028 = np.corrcoef(my_eq_Uphi[epoch2:].flatten(), ref_eq_Uphi[epoch2,:].flatten())[0,1]
ax12.set_title(r'UCM ('+str(np.round(corr2023,3))+', '+str(np.round(corr2028,3))+')')

my_model= readPlotmodelhdf5(str(pygeodyn_results)+'USTHB_Current_computation.hdf5')
measure = my_model.measures['U']
components=["th","ph"]
griddata = measure.computeRThetaPhiData(rC, mytheta, myphi,components=components,computeallrealisation=False,irealisation=-1)
my_Uphi = griddata["ph"]

my_eq_Uphi = np.zeros((loc_end-loc_start, Nph))
for i in range(Nph):
    my_eq_Uphi[:,i] = my_Uphi[loc_start:loc_end,lat_request,i]#- ref_Uphi[loc_start:loc_end,lat_request,i]
ax13.imshow(np.flipud(my_eq_Uphi),cmap=cmap, extent=[0, 360, TL_start, TL_end], aspect='auto', vmin=-color_speed, vmax=color_speed)
corr2023 = np.corrcoef(my_eq_Uphi[epoch1,:].flatten(), ref_eq_Uphi[epoch1,:].flatten())[0,1]
corr2028 = np.corrcoef(my_eq_Uphi[epoch2:].flatten(), ref_eq_Uphi[epoch2,:].flatten())[0,1]
ax13.set_title(r'USTHB ('+str(np.round(corr2023,3))+', '+str(np.round(corr2028,3))+')')

my_model= readPlotmodelhdf5(str(pygeodyn_results)+'WHU_Current_computation.hdf5')
measure = my_model.measures['U']
components=["th","ph"]
griddata = measure.computeRThetaPhiData(rC, mytheta, myphi,components=components,computeallrealisation=False,irealisation=-1)
my_Uphi = griddata["ph"]

my_eq_Uphi = np.zeros((loc_end-loc_start, Nph))
for i in range(Nph):
    my_eq_Uphi[:,i] = my_Uphi[loc_start:loc_end,lat_request,i]#- ref_Uphi[loc_start:loc_end,lat_request,i]
ax14.imshow(np.flipud(my_eq_Uphi),cmap=cmap, extent=[0, 360, TL_start, TL_end], aspect='auto', vmin=-color_speed, vmax=color_speed)
corr2023 = np.corrcoef(my_eq_Uphi[epoch1,:].flatten(), ref_eq_Uphi[epoch1,:].flatten())[0,1]
corr2028 = np.corrcoef(my_eq_Uphi[epoch2:].flatten(), ref_eq_Uphi[epoch2,:].flatten())[0,1]
ax14.set_title(r'WHU ('+str(np.round(corr2023,3))+', '+str(np.round(corr2028,3))+')')

ax1.set_yticks([2020, 2025, 2030])
ax4.set_yticks([2020, 2025, 2030])
ax7.set_yticks([2020, 2025, 2030])
ax10.set_yticks([2020, 2025, 2030])
ax13.set_yticks([2020, 2025, 2030])

ax1.set_ylabel(r'Time (Year)')
ax4.set_ylabel(r'Time (Year)')
ax7.set_ylabel(r'Time (Year)')
ax10.set_ylabel(r'Time (Year)')
ax13.set_ylabel(r'Time (Year)')

ax13.set_xlabel(r'Longitude (Degrees)')
ax14.set_xlabel(r'Longitude (Degrees)')
ax12.set_xlabel(r'Longitude (Degrees)')

ax13.set_xticks([0, 90, 180, 270, 360])
ax14.set_xticks([0, 90, 180, 270, 360])
ax12.set_xticks([0, 90, 180, 270, 360])


ax1.grid()
ax2.grid()
ax3.grid()
ax4.grid()
ax5.grid()
ax6.grid()
ax7.grid()
ax8.grid()
ax9.grid()
ax10.grid()
ax11.grid()
ax12.grid()
ax13.grid()
ax14.grid()

cax = axes[14]
cax.axis('off')  # turn off map projection
# Create a colorbar axis inside it
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
cbar_ax = inset_axes(cax, width="40%", height="80%", loc='center')

# --- Create the shared colorbar ---
norm = mcolors.Normalize(vmin=-color_speed, vmax=color_speed)
sm = cm.ScalarMappable(cmap=cmap, norm=norm)
sm.set_array([])

fig.colorbar(sm, cax=cbar_ax, orientation='vertical', label='Zonal wind speed (m/s)')


plt.show()

Figure 6(a) Plots of flow for DTU and Kalmag for Lsv=13

In [ ]:
time_step= 2030.0
loc = np.where(PlotModels['BGS']['Time'] == time_step)[0][0]
Nph = 180
Nth = 91
x = np.linspace(-np.pi,np.pi,Nph)
y = np.linspace(-np.pi/2+0.01,np.pi/2-0.01,Nth)
X,Y = np.meshgrid(x,y)
myphi = x
mytheta = np.pi/2-y
phi1= x*180/np.pi
lat = y*180/np.pi

#####
myfontsize=20
vm=20
dv=0.1
Nlev = int((2*vm/dv)+1)
levels = np.linspace(-vm,vm,Nlev)

sum_diff_map = np.zeros((Nth, Nph))

my_epoch = loc
fig = plt.figure(figsize=(12,11))

PlotModels1 = webgeodyn.models.Models()
ref_model= readPlotmodelhdf5(str(pygeodyn_results)+'Huber_Current_computation.hdf5')
measure = ref_model.measures['U']
components=["th","ph"]
griddata = measure.computeRThetaPhiData(rC, mytheta, myphi,components=components,computeallrealisation=False,irealisation=-1)
ref_Uphi = griddata["ph"]
ref_Uth = griddata["th"]
ref_dated_Uphi = ref_Uphi[my_epoch,:,:]
ref_dated_Uth =  (-1)*ref_Uth[my_epoch,:,:]

## DTU L=8
PlotModels1 = webgeodyn.models.Models()
my_model= readPlotmodelhdf5(str(pygeodyn_results)+'DTU_Current_computation.hdf5')
measure = my_model.measures['U']
components=["th","ph"]
griddata = measure.computeRThetaPhiData(rC, mytheta, myphi,components=components,computeallrealisation=False,irealisation=-1)
my_Uphi = griddata["ph"]
my_Uth = griddata["th"]
my_dated_Uphi = my_Uphi[my_epoch,:,:]
my_dated_Uth =  (-1)*my_Uth[my_epoch,:,:]
U_abs = np.sqrt(my_dated_Uphi**2+ my_dated_Uth**2)
PHI, THETA = np.mgrid[-180:180:180j, -90:90:91j]
lw = 5*U_abs / (0.5*np.max(np.sqrt(ref_dated_Uphi**2+ ref_dated_Uth**2)))

#fig=plt.figure(figsize=(6,3))
ax1 = fig.add_subplot(3,2,1, projection=ccrs.Mollweide())
ax1_ = ax1.contourf(phi1, lat, my_dated_Uphi, levels, transform=ccrs.PlateCarree(),cmap='RdBu_r',extend='both')
ax1.coastlines(resolution='110m')
ax1.streamplot(phi1, lat, my_dated_Uphi, my_dated_Uth,  transform=ccrs.PlateCarree(), density=[1.5, 0.75], color='k', linewidth = lw)
ax1.gridlines()
ax1.set_title(r'DTU L=8 2030')

modeldtu8=U_abs
modeldtu8_phi= my_dated_Uphi
modeldtu8_th = my_dated_Uth

## TU_Berlin
PlotModels1 = webgeodyn.models.Models()
my_model= readPlotmodelhdf5(str(pygeodyn_results)+'TU_Berlin_Current_computation.hdf5')
measure = my_model.measures['U']
components=["th","ph"]
griddata = measure.computeRThetaPhiData(rC, mytheta, myphi,components=components,computeallrealisation=False,irealisation=-1)
my_Uphi = griddata["ph"]
my_Uth = griddata["th"]
my_dated_Uphi = my_Uphi[my_epoch,:,:]
my_dated_Uth =  (-1)*my_Uth[my_epoch,:,:] 
U_abs = np.sqrt(my_dated_Uphi**2+ my_dated_Uth**2)
PHI, THETA = np.mgrid[-180:180:180j, -90:90:91j]
lw = 5*U_abs / (0.5*np.max(np.sqrt(ref_dated_Uphi**2+ ref_dated_Uth**2)))

#fig=plt.figure(figsize=(6,3))
ax2 = fig.add_subplot(3,2,2,projection=ccrs.Mollweide())
ax2.coastlines(resolution='110m')
ax2.contourf(phi1, lat, my_dated_Uphi, levels, transform=ccrs.PlateCarree(),cmap='RdBu_r',extend='both')
ax2.streamplot(phi1, lat, my_dated_Uphi, my_dated_Uth,  transform=ccrs.PlateCarree(),  density=[1.5, 0.75], color='k', linewidth = lw)
ax2.gridlines()
ax2.set_title(r'TUB L=8 2030')

modeltub8=U_abs
modeltub8_phi= my_dated_Uphi
modeltub8_th = my_dated_Uth

## DTU L=13
PlotModels1 = webgeodyn.models.Models()
my_model= readPlotmodelhdf5(str(pygeodyn_results)+'DTU_13_Current_computation.hdf5')
measure = my_model.measures['U']
components=["th","ph"]
griddata = measure.computeRThetaPhiData(rC, mytheta, myphi,components=components,computeallrealisation=False,irealisation=-1)
my_Uphi = griddata["ph"]
my_Uth = griddata["th"]
my_dated_Uphi = my_Uphi[my_epoch,:,:]
my_dated_Uth =  (-1)*my_Uth[my_epoch,:,:]
U_abs = np.sqrt(my_dated_Uphi**2+ my_dated_Uth**2)
PHI, THETA = np.mgrid[-180:180:180j, -90:90:91j]
lw = 5*U_abs / (0.5*np.max(np.sqrt(ref_dated_Uphi**2+ ref_dated_Uth**2)))

#fig=plt.figure(figsize=(6,3))
ax3 = fig.add_subplot(3,2,3,projection=ccrs.Mollweide())
ax3_ = ax3.contourf(phi1, lat, my_dated_Uphi, levels, transform=ccrs.PlateCarree(),cmap='RdBu_r',extend='both')
ax3.coastlines(resolution='110m')
ax3.streamplot(phi1, lat, my_dated_Uphi, my_dated_Uth,  transform=ccrs.PlateCarree(), density=[1.5, 0.75], color='k', linewidth = lw)
ax3.gridlines()
ax3.set_title(r'DTU L=13 2030')

modeldtu13=U_abs
modeldtu13_phi= my_dated_Uphi
modeldtu13_th = my_dated_Uth
lw = 5*(modeldtu13-modeldtu8)  / (0.5*np.max(np.sqrt(ref_dated_Uphi**2+ ref_dated_Uth**2)))

ax5 = fig.add_subplot(3,2,5,projection=ccrs.Mollweide())
ax5_ = ax5.contourf(phi1, lat, modeldtu13_phi-modeldtu8_phi, levels, transform=ccrs.PlateCarree(),cmap='RdBu_r',extend='both')
ax5.coastlines(resolution='110m')
ax5.streamplot(phi1, lat, modeldtu13_phi-modeldtu8_phi, modeldtu13_th-modeldtu8_th,  transform=ccrs.PlateCarree(), density=[1.5, 0.75], color='k', linewidth = lw)
ax5.gridlines()
ax5.set_title(r'DTU 2030 L=13 map - L=8 map')


## TU_Berlin
PlotModels1 = webgeodyn.models.Models()
my_model= readPlotmodelhdf5(str(pygeodyn_results)+'TU_Berlin_13_Current_computation.hdf5')
measure = my_model.measures['U']
components=["th","ph"]
griddata = measure.computeRThetaPhiData(rC, mytheta, myphi,components=components,computeallrealisation=False,irealisation=-1)
my_Uphi = griddata["ph"]
my_Uth = griddata["th"]
my_dated_Uphi = my_Uphi[my_epoch,:,:]
my_dated_Uth =  (-1)*my_Uth[my_epoch,:,:]
U_abs = np.sqrt(my_dated_Uphi**2+ my_dated_Uth**2)
PHI, THETA = np.mgrid[-180:180:180j, -90:90:91j]
lw = 5*U_abs / (0.5*np.max(np.sqrt(ref_dated_Uphi**2+ ref_dated_Uth**2)))

ax4 = fig.add_subplot(3,2,4, projection=ccrs.Mollweide())
ax4.coastlines(resolution='110m')
ax4.contourf(phi1, lat, my_dated_Uphi, levels, transform=ccrs.PlateCarree(),cmap='RdBu_r',extend='both')
ax4.streamplot(phi1, lat, my_dated_Uphi, my_dated_Uth,  transform=ccrs.PlateCarree(),  density=[1.5, 0.75], color='k', linewidth = lw)
ax4.gridlines()
ax4.set_title(r'TUB L=13 2030')

modeltub13=U_abs
modeltub13_phi= my_dated_Uphi
modeltub13_th = my_dated_Uth

lw = 5*(modeltub13-modeltub8)  / (0.5*np.max(np.sqrt(ref_dated_Uphi**2+ ref_dated_Uth**2)))

ax6 = fig.add_subplot(3,2,6,projection=ccrs.Mollweide())
ax6_ = ax6.contourf(phi1, lat, modeltub13_phi-modeltub8_phi, levels, transform=ccrs.PlateCarree(),cmap='RdBu_r',extend='both')
ax6.coastlines(resolution='110m')
ax6.streamplot(phi1, lat, modeltub13_phi-modeltub8_phi, modeltub13_th-modeltub8_th,  transform=ccrs.PlateCarree(), density=[1.5, 0.75], color='k', linewidth = lw)
ax6.gridlines()
ax6.set_title(r'TUB 2030 L=13 map - L=8 map')


#plt.colorbar(ax_)
#plt.show()

plt.show()


In [ ]:
# correlations for the L=13 maps against the Huber L=8 reference
corr = np.corrcoef(modeldtu13_phi.flatten(), ref_Uphi[my_epoch,:,:].flatten())[0,1]
print(corr)

corr = np.corrcoef(modeltub13_phi.flatten(), ref_Uphi[my_epoch,:,:].flatten())[0,1]
print(corr)

Figure 6(b) Spectra of the SV and U for the DTU and Kalmag for Lsv=13

In [ ]:
PlotModels = readmodelhdf5(str(pygeodyn_results)+'DTU_13_Current_computation.hdf5', 'DTU_13', 'red', 'x','-')
PlotModels = readmodelhdf5(str(pygeodyn_results)+'TU_Berlin_13_Current_computation.hdf5', 'TUB_13', 'blue', 'x','-')


fig_SV, ((ax1), (ax2)) = plt.subplots(2, 1, layout='constrained', figsize=(8,8))
SV_spec = []
ER_spec = []

loc = np.where(PlotModels['DTU']['Time'] == 2030.0)[0]
lmax = 8
x_SV = np.array(np.linspace(1,lmax, lmax))
array2D_SV = PlotModels['DTU']['SV'][loc][0]
DTU_spec = computeSVSpectra(lmax, array2D_SV)
array2D_ER = PlotModels['DTU']['ER'][loc][0]
DTU_ERspec = computeSVSpectra(lmax, array2D_ER)
ax1.plot(x_SV, np.array(DTU_spec), label = 'DTU, L=8', color = PlotModels['DTU']['Color'], ls = '-', marker = PlotModels['DTU']['Marker'],  markersize=5, lw=1)
ax1.plot(x_SV, np.array(DTU_ERspec), label = 'DTU, L=8', color = PlotModels['DTU']['Color'], ls = '--', marker = PlotModels['DTU']['Marker'],  markersize=5, lw=1)

array2D_SV = PlotModels['TUB']['SV'][loc][0]
TUB_spec = computeSVSpectra(lmax, array2D_SV)
array2D_ER = PlotModels['TUB']['ER'][loc][0]
TUB_ERspec = computeSVSpectra(lmax, array2D_ER)
ax1.plot(x_SV, np.array(TUB_spec), label = 'TUB, L=8', color = PlotModels['TUB']['Color'], ls = '-', marker = PlotModels['TUB']['Marker'],  markersize=5, lw=1)
ax1.plot(x_SV, np.array(TUB_ERspec), label = 'TUB, L=8', color = PlotModels['TUB']['Color'], ls = '--', marker = PlotModels['TUB']['Marker'],  markersize=5, lw=1)

lmax = 13
x_SV = np.array(np.linspace(1,lmax, lmax))
array2D_SV = PlotModels['DTU_13']['SV'][loc][0]
DTU13_spec = computeSVSpectra(lmax, array2D_SV)
array2D_ER = PlotModels['DTU_13']['ER'][loc][0]
DTU13_ERspec = computeSVSpectra(lmax, array2D_ER)
ax1.plot(x_SV, np.array(DTU13_spec), label = 'DTU, L=13', color = PlotModels['DTU_13']['Color'], ls = '-', marker = PlotModels['DTU_13']['Marker'],  markersize=5, lw=1)
ax1.plot(x_SV, np.array(DTU13_ERspec), label = 'DTU, L=13', color = PlotModels['DTU_13']['Color'], ls = '--', marker = PlotModels['DTU_13']['Marker'],  markersize=5, lw=1)

array2D_SV = PlotModels['TUB_13']['SV'][loc][0]
TUB13_spec = computeSVSpectra(lmax, array2D_SV)
array2D_ER = PlotModels['TUB_13']['ER'][loc][0]
TUB13_ERspec = computeSVSpectra(lmax, array2D_ER)
ax1.plot(x_SV, np.array(TUB13_spec), label = 'TUB, L=13', color = PlotModels['TUB_13']['Color'], ls = '-', marker = PlotModels['TUB_13']['Marker'],  markersize=5, lw=1)
ax1.plot(x_SV, np.array(TUB13_ERspec), label = 'TUB, L=13', color = PlotModels['TUB_13']['Color'], ls = '--', marker = PlotModels['TUB_13']['Marker'],  markersize=5, lw=1)

#lmax = 8
#x_SV = np.array(np.linspace(1,lmax, lmax))
#ax1.plot(x_SV, np.array(DTU_spec-DTU13_spec[:lmax]), label = 'DTU difference', color = PlotModels['DTU']['Color'], ls = ':', marker = PlotModels['DTU_13']['Marker'],  markersize=5, lw=1)
#ax1.plot(x_SV, np.array(TUB_spec-TUB13_spec[:lmax]), label = 'TU_Berlin difference', color = PlotModels['TU_Berlin']['Color'], ls = ':', marker = PlotModels['TU_Berlin_13']['Marker'],  markersize=5, lw=1)
#lmax = 13

ax1.set_title('SV and ER in January 2030', fontsize=14)
ax1.set_xlabel('Spherical Harmonic Degree', fontsize=12)
ax1.set_ylabel('Spectral Energy $(nT/yr)^2$', fontsize=12)
#ax1.set_xticks(np.linspace(2,12, 6))
ax1.semilogy()
ax1.axis([0, lmax+1, 0.01, 10000])
ax1.set_box_aspect(1)
    
handles,labels = ax1.get_legend_handles_labels()
#ax1.legend( (handles[0], handles[2], handles[4], handles[6], handles[8], handles[10], Line2D([0], [0], color='tab:cyan'), Line2D([0], [0], color='tab:pink'),  Line2D([0], [0], color='green', ls= '--'), Line2D([0], [0], color='green', ls= 'dotted'), Line2D([0], [0], color='k'), Line2D([0], [0], ls='--', color='k'), Line2D([0], [0], ls='dotted', color='k')), ['Kalmag, 71p', 'Kalmag, 50p', 'Kalmag, 0p', 'Kalmag, Neutral_top1', 'Kalmag, Stable_top1', 'Kalmag, S1$^{\gamma}$', 'CovObs.x2, Neutral_top1', 'CHAOS-7.16, Neutral_top1', 'Kalmag, $L_{SV}=13$', 'Kalmag, $L_{SV}=18$', 'SV', 'ER', 'Difference from SV model'], loc='center left', bbox_to_anchor=(1, 0.5))
ax1.grid()
#plt.show()


lmax = 15
x_U = np.array(np.linspace(1,lmax, lmax))
#fig_U, (ax1) = plt.subplots(1, 1, layout='constrained', sharey=True, figsize=(7,5))
U_spec = []

x_SV = np.array(np.linspace(1,lmax, lmax))
array2D_U = PlotModels['DTU']['U'][loc][0]
DTU_spec = computeSVSpectra(lmax, array2D_U)
ax2.plot(x_U, np.array(DTU_spec), label = 'DTU, L=8', color = PlotModels['DTU']['Color'], ls = '-', marker = PlotModels['DTU']['Marker'],  markersize=5, lw=1)

array2D_U = PlotModels['TUB']['U'][loc][0]
TUB_spec = computeSVSpectra(lmax, array2D_U)
ax2.plot(x_U, np.array(TUB_spec), label = 'TUB, L=8', color = PlotModels['TUB']['Color'], ls = '-', marker = PlotModels['TUB']['Marker'],  markersize=5, lw=1)

array2D_U = PlotModels['ITES']['U'][loc][0]
ITES_spec = computeSVSpectra(lmax, array2D_U)
ax2.plot(x_U, np.array(ITES_spec), label = 'ITES, L=8', color = PlotModels['ITES']['Color'], ls = '-', marker = PlotModels['ITES']['Marker'],  markersize=5, lw=1)

array2D_U = PlotModels['DTU_13']['U'][loc][0]
DTU13_spec = computeSVSpectra(lmax, array2D_U)
ax2.plot(x_U, np.array(DTU13_spec), label = 'DTU, L=13', color = PlotModels['DTU']['Color'], ls = '-', marker = PlotModels['DTU_13']['Marker'],  markersize=5, lw=1)

array2D_U = PlotModels['TUB_13']['U'][loc][0]
TUB13_spec = computeSVSpectra(lmax, array2D_U)
ax2.plot(x_U, np.array(TUB13_spec), label = 'TUB, L=13', color = PlotModels['TUB']['Color'], ls = '-', marker = PlotModels['TUB_13']['Marker'],  markersize=5, lw=1)

ax2.set_title('U in January 2030', fontsize=14)
ax2.set_xlabel('Spherical Harmonic Degree', fontsize=12)
ax2.set_ylabel('Spectral Energy $(km/yr)^2$', fontsize=12)
ax2.set_xticks(np.linspace(0,14, 8))
ax2.semilogy()
ax2.axis([0, lmax+1, 0.5, 200])
ax2.set_box_aspect(1)
    
#ax1.legend( (handles[0], handles[2], handles[4], handles[6], handles[8], handles[10], Line2D([0], [0], color='tab:cyan'), Line2D([0], [0], color='tab:pink'),  Line2D([0], [0], color='green', ls= '--'), Line2D([0], [0], color='green', ls= 'dotted'), Line2D([0], [0], color='k'), Line2D([0], [0], ls='--', color='k'), Line2D([0], [0], ls='dotted', color='k')), ['Kalmag, 71p', 'Kalmag, 50p', 'Kalmag, 0p', 'Kalmag, Neutral_top1', 'Kalmag, Stable_top1', 'Kalmag, S1$^{\gamma}$', 'CovObs.x2, Neutral_top1', 'CHAOS-7.16, Neutral_top1', 'Kalmag, $L_{SV}=13$', 'Kalmag, $L_{SV}=18$', 'SV', 'ER', 'Difference from SV model'], loc='center left', bbox_to_anchor=(1, 0.5))
ax2.grid()
#ax2.grid()

handles,labels = ax2.get_legend_handles_labels()
ax2.legend(handles, labels, loc = 'best')#bbox_to_anchor=(1,0.5), loc='center left', fontsize=10.5)
plt.show()
